# Importing packages, context and root

In [1]:
from __future__ import division
from hail import *
import collections
import itertools
import pandas as pd
import csv
import tempfile
from pyspark.sql.types import DoubleType
from pyspark.sql import SQLContext
from pprint import pprint
root = '...'
hc = HailContext(log="/home/hail/hail.log")


Running on Apache Spark version 2.0.2
SparkUI available at http://10.128.0.4:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.1-0d9e264


# PART 0: Quality control

This first part describes the QC of the raw vcf. The code is in ``Hail``.

Basic variant-level QC that we use. This is the general QC applied across the different datasets, but there are additional sample-level filters applied to each dataset, which are not reported here

In [2]:
## raw_vds: input vds
## out_vds: output vds
## samp_excl_list: list of samples to exclude

def basic_qc(raw_vds,out_vds,samp_excl_list=[])
    raw_vds = hc.read(raw_vds)
    
    if samp_excl_list:
        vdsT1 = (raw_vds
               .filter_samples_list(samp_excl_list, keep=False))
    else:
        vdsT1 = raw_vds
        
    (vdsT1
    .split_multi()
    .variant_qc()
    .filter_variants_expr('va.qc.AC > 0 && va.filters.isEmpty()', keep=True)
    .filter_genotypes(
         """
         (g.isHomRef && (g.ad[0] / g.dp < 0.8 || g.gq < 20 || g.dp < 20)) ||
         (g.isHomVar && (g.ad[1] / g.dp < 0.8 || g.pl[0] < 20 || g.dp < 20)) ||
         (g.isHet && ( (g.ad[0] + g.ad[1]) / g.dp < 0.8 || g.ad[1] / g.dp < 0.20 || g.pl[0] < 20 || g.dp < 20))
         """, keep = False)
    .filter_variants_expr('va.qc.pHWE > 0.000001 && va.qc.callRate > 0.80 && v.contig != "X" && v.contig != "Y" && v.contig != "MT"', keep=True)
    .hardcalls()
    .repartition(5000,shuffle=False)
    .write(out_vds))

# PART 1: Calculate principal components

This first part describes the steps used to calculate PCs. The code is in ``Hail``

Import 1000 genomes dataset

In [4]:
onekpg = hc.read(root + 'ALL.1KG.qc.hardcalls.vds')

Put toghether the dataset and 1000 genome to save global PCs

In [23]:
### vds: input vds
### root: root where to export the PCs
### samp_excl_list: list of samples to exclude

def pc_1000G(vds,root,samp_excl_list=[]):
    
    if samp_excl_list:
        vdsT1 = (vds
               .filter_samples_list(samp_excl_list, keep=False)
               .variant_qc()
               .filter_variants_expr('va.qc.AC >  0', keep=True))
    else:
        vdsT1 = (vds
               .variant_qc()
               .filter_variants_expr('va.qc.AC >  0', keep=True))
    
    vdsT2 = (vdsT1
        .filter_intervals(Interval.parse('6:25M-35M'), keep=False)
        .filter_intervals(Interval.parse('8:7M-15M'), keep=False)
        .filter_variants_expr('va.qc.AF > 0.05 && va.qc.callRate > 0.98 && va.qc.pHWE > 0.001')
        .ld_prune(r2=0.2, window=1000000, num_cores=400)
        .annotate_samples_expr('sa = {}'))
    
    
    onekpgQ = (onekpg
         .annotate_samples_expr('sa = {}'))

    ID1=(onekpgQ.sample_ids)

    

    ID2=(vdsT2.sample_ids)

    # Remove individuals in both datasets
    intsect = list(set(ID1) & set(ID2))

    (vdsT2
    .filter_samples_list(intsect, keep=False)
    .join(onekpgQ)
    .pca(scores = 'sa.pca_1kg')
    .export_samples( root + '/pca_1kg.tsv','id=s, sa.pca_1kg.*'))

### Annotate the studies wiht VEP

In [ ]:
## vds: input vds
## out: out annotated vds
def run_VEP(vds,out_vds):
    (vds
    .annotate_variants_expr('va = drop(va, vep)')
    .vep(config='/vep/vep-gcloud.properties', root='va.vep')
    .write(out_vds, overwrite=True))

# PART 2: Annotations and creating burden scores for analysis

This first part describes the steps used for creating the scores used in the analysis. The code is mostly in ``Hail``, but there are some parts in R.

Read datasets containing other annotations. These dataset do not contain inidividual-level data and are shared.

- `dbNSFP` dbNSFP used for annotation of missense damaging,
- `lethal` dominant lethal,
- `clinvar` curated clinvar variants
- `MPC` missense badness score from Kaitlin samocha
- `high_qual_intervals` intervals with good coverage across all exomes captures kits

In [3]:
dbNSFP = hc.read(root + 'dbNSFP_3.2a_variant.filtered.allhg19_nodup.vds')
lethal = hc.import_table(root + 'lethal_clean.txt',impute=True).key_by('Variant')
clinvar = hc.import_table(root + 'clinvar_clean.txt',impute=True).key_by('Variant')
MPC = hc.import_table(root + 'fordist_constraint_official_mpc_values.txt.gz',impute=True,
                      types = {'chrom': TString(),
                                            'pos': TInt(),
                                            'ref': TString(),
                                            'alt': TString()}).annotate('variant = Variant(chrom, pos, ref, alt)').key_by('variant')

high_qual_intervals = (hc.import_table(root + 'high_quality_exome_regions.bed',no_header=True, impute=True)
.annotate('interval = Interval(Locus(f0, f1), Locus(f0, f2))')
.key_by('interval'))

Attach the dataset and define if variant is in clinvar or lethal, define allele frequencies categories and URV, that is, variants not observed in any of the other dataset. Define also variants that are doubletons in the study but not observed in any other dataset. 
Note that vds 'annovdslist' are not shared since the include individual-level data. However we made available the variants in the github repository.

In [22]:
## vds: input vds
## annovdslist: list of vds used to filter the current vds 
##        (URV variants are defined as singletons in the vds and not seen in any of the other vds passed in annovdslist)
## samp_excl_list: list of samples to exclude


def annot_phase1(vds,annovdslist,names_annovdslist,samp_excl_list=[]):
    
    if samp_excl_list:
        vdsT1 = (vds
               .filter_samples_list(samp_excl_list, keep=False)
               .variant_qc()
               .filter_variants_expr('va.qc.AC >  0', keep=True)
               .annotate_variants_vds(dbNSFP,root='va.dbNSFP'))
    else:
        vdsT1 = (vds
               .variant_qc()
               .filter_variants_expr('va.qc.AC >  0', keep=True)
               .annotate_variants_vds(dbNSFP,root='va.dbNSFP')) 
    
    tt = names_annovdslist[0]
    tts=tt.split('.')
    vdsT2 = (vdsT1
             .annotate_variants_vds(annovdslist[0],expr='va.%s=isDefined(vds)' % names_annovdslist[0]))

    for num, vdsL in enumerate(annovdslist[1:len(annovdslist)]):  
        vdsT2 = (vdsT2
                .annotate_variants_vds(vdsL,expr='va.%s=isDefined(vds)' % names_annovdslist[num+1]))
    
    expression = """
        va.freq.AF001 = (va.qc.AF < 0.001),
        va.freq.DOUBLE = (va.qc.AC == 2),
        va.freq.SING = (va.nNonRef == 1),
        va.freq.URV = (va.nNonRef == 1 && {0} ),
        va.freq.DOUBLEURV = (va.qc.AC < 3 && {1} )
        """ 
    
    annot1vds=(vdsT2
    .annotate_variants_table(lethal,expr='va.lethal = table')
    .annotate_variants_table(clinvar,expr='va.clinvar = table')
    .annotate_variants_table(MPC,expr='va.MPC = table.MPC')
    .annotate_variants_expr(
        """
        va.nNonRef = gs.filter(g => g.isCalledNonRef).count()
        """)
    .annotate_variants_expr(
        expression.format("!va." + " && !va.".join(names_annovdslist),"!va." + " && !va.".join(names_annovdslist))
    ))
    
    return(annot1vds)


Create VEP-based annotations: 
- `LoF`: frameshift_variant, transcript_ablation, splice_acceptor_variant, splice_donor_variant, stop_gained
- `Missense`: missense_variant,inframe_insertion,inframe_deletion
- `LOFdamaging`: Lof + missense damaging. Where `damaging` is defined according to the number of different damaging algorithms from dbNSFP.

In [24]:
## vds: input vds

def annot_phase2(vds):
    
    annot2vds = (vds 
    .annotate_variants_expr('va.geneann.transcript_canonicals = va.vep.transcript_consequences.filter(tc => tc.canonical == 1)')
    .annotate_variants_expr(
        """
        va.geneann.most_severe_csq = 
        let x = va.geneann.transcript_canonicals.map(y => y.consequence_terms.toSet) in 
        if (isMissing(x)) va.vep.most_severe_consequence
        else if (x.exists(tc => tc.contains("transcript_ablation"))) "transcript_ablation"
        else if (x.exists(tc => tc.contains("splice_acceptor_variant"))) "splice_acceptor_variant"
        else if (x.exists(tc => tc.contains("splice_donor_variant"))) "splice_donor_variant"
        else if (x.exists(tc => tc.contains("stop_gained"))) "stop_gained"
        else if (x.exists(tc => tc.contains("frameshift_variant"))) "frameshift_variant"
        else if (x.exists(tc => tc.contains("stop_lost"))) "stop_lost"
        else if (x.exists(tc => tc.contains("start_lost"))) "start_lost"
        else if (x.exists(tc => tc.contains("transcript_amplification"))) "transcript_amplification"
        else if (x.exists(tc => tc.contains("inframe_insertion"))) "inframe_insertion"
        else if (x.exists(tc => tc.contains("inframe_deletion"))) "inframe_deletion"
        else if (x.exists(tc => tc.contains("missense_variant"))) "missense_variant"
        else if (x.exists(tc => tc.contains("protein_altering_variant"))) "protein_altering_variant"
        else if (x.exists(tc => tc.contains("splice_region_variant"))) "splice_region_variant"
        else if (x.exists(tc => tc.contains("incomplete_terminal_codon_variant"))) "incomplete_terminal_codon_variant"
        else if (x.exists(tc => tc.contains("stop_retained_variant"))) "stop_retained_variant"
        else if (x.exists(tc => tc.contains("synonymous_variant"))) "synonymous_variant"
        else if (x.exists(tc => tc.contains("coding_sequence_variant"))) "coding_sequence_variant"
        else if (x.exists(tc => tc.contains("mature_miRNA_variant"))) "mature_miRNA_variant"
        else if (x.exists(tc => tc.contains("5_prime_UTR_variant"))) "5_prime_UTR_variant" 
        else if (x.exists(tc => tc.contains("3_prime_UTR_variant"))) "3_prime_UTR_variant"
        else if (x.exists(tc => tc.contains("non_coding_transcript_exon_variant"))) "non_coding_transcript_exon_variant"
        else if (x.exists(tc => tc.contains("intron_variant"))) "intron_variant"
        else if (x.exists(tc => tc.contains("NMD_transcript_variant"))) "NMD_transcript_variant"
        else if (x.exists(tc => tc.contains("non_coding_transcript_variant"))) "non_coding_transcript_variant"
        else if (x.exists(tc => tc.contains("upstream_gene_variant"))) "upstream_gene_variant"
        else if (x.exists(tc => tc.contains("downstream_gene_variant"))) "downstream_gene_variant"
        else if (x.exists(tc => tc.contains("TFBS_ablation"))) "TFBS_ablation"
        else if (x.exists(tc => tc.contains("TFBS_amplification"))) "TFBS_amplification"
        else if (x.exists(tc => tc.contains("TF_binding_site_variant"))) "TF_binding_site_variant"
        else if (x.exists(tc => tc.contains("regulatory_region_ablation"))) "regulatory_region_ablation"
        else if (x.exists(tc => tc.contains("regulatory_region_amplification"))) "regulatory_region_amplification"
        else if (x.exists(tc => tc.contains("feature_elongation"))) "feature_elongation"
        else if (x.exists(tc => tc.contains("regulatory_region_variant"))) "regulatory_region_variant"
        else if (x.exists(tc => tc.contains("feature_truncation"))) "feature_truncation"
        else if (x.exists(tc => tc.contains("intergenic_variant"))) "intergenic_variant"
        else va.vep.most_severe_consequence
        """)
    .annotate_variants_expr(
        """
        va.geneann.gene = 
        if (va.geneann.transcript_canonicals.exists(tc => tc.consequence_terms.toSet.contains(va.geneann.most_severe_csq))) (va.geneann.transcript_canonicals.find(tc => tc.canonical == 1 && tc.consequence_terms.toSet.contains(va.geneann.most_severe_csq)).gene_symbol)
        else va.vep.transcript_consequences.find(tc => tc.consequence_terms.toSet.contains(va.geneann.most_severe_csq)).gene_symbol
        """)
    .annotate_variants_expr(
        """
        va.geneann.LOFTEElof_flags = 
        if (va.geneann.transcript_canonicals.exists(tc => tc.consequence_terms.toSet.contains(va.geneann.most_severe_csq))) (va.geneann.transcript_canonicals.find(tc => tc.canonical == 1 && tc.consequence_terms.toSet.contains(va.geneann.most_severe_csq)).lof)
        else va.vep.transcript_consequences.find(tc => tc.consequence_terms.toSet.contains(va.geneann.most_severe_csq)).lof_flags
        """)
    .annotate_variants_expr('va.geneann.LOFTEElof = (if (va.geneann.LOFTEElof_flags=="HC") 1 else 0)')
    .annotate_variants_expr(
        """
        va.geneann.damascore = (if ("D" ~ va.dbNSFP.SIFT_pred) 1 else 0) + (if ("D" ~ va.dbNSFP.PROVEAN_pred) 1 else 0) + (if ("D" ~ va.dbNSFP.Polyphen2_HDIV_pred) 1 else 0) + (if ("D" ~ va.dbNSFP.Polyphen2_HVAR_pred) 1 else 0) + (if ("D" ~ va.dbNSFP.LRT_pred) 1 else 0) + (if ("[HM]" ~ va.dbNSFP.MutationAssessor_pred) 1 else 0) + (if ("[AD]" ~ va.dbNSFP.MutationTaster_pred) 1 else 0)
        """)
    .annotate_variants_expr(
        """
        va.geneann.LOF = (if (va.geneann.most_severe_csq == "frameshift_variant" || va.geneann.most_severe_csq == "transcript_ablation" || va.geneann.most_severe_csq == "splice_acceptor_variant" || va.geneann.most_severe_csq == "splice_donor_variant" || va.geneann.most_severe_csq == "stop_gained") 1 else 0)
        """)
    .annotate_variants_expr(
        """
        va.geneann.missense = (if (va.geneann.most_severe_csq == "missense_variant" || va.geneann.most_severe_csq == "inframe_insertion" || va.geneann.most_severe_csq == "inframe_deletion") 1 else 0)
        """)
    .annotate_variants_expr(
        """
        va.geneann.LOFdamaging = (if (va.geneann.LOF==1 || (va.geneann.missense==1 && va.geneann.damascore >= 7)) 1 else 0),
        va.geneann.dama1 = (if ((va.geneann.missense==1 && va.geneann.damascore >= 1)) 1 else 0), 
        va.geneann.dama3 = (if ((va.geneann.missense==1 && va.geneann.damascore >= 3)) 1 else 0), 
        va.geneann.dama7 = (if ((va.geneann.missense==1 && va.geneann.damascore >= 7)) 1 else 0),
        va.geneann.damaMPC = (if ((va.geneann.missense==1 && va.MPC >= 2)) 1 else 0),
        va.geneann.synonymous = (if (va.geneann.most_severe_csq == "synonymous_variant") 1 else 0)
        """))
    return(annot2vds)

Here we import a bunch of genesets. The input file has this format: the first column is the gene name. Each column is a different gene set. Each cell is 1 if the gene is in that gene set and 0 if it is not.
We create a dictionary that looks like: `{'RAB40C': ['pLO09', 'essential_mice2', 'ALL'],'AL022328.1': []}`.
We also read all the genes names and not in the geneset list and add those as well, with missing geneset = `[]`

In [3]:
sqlContext = SQLContext(hc.sc)
file=sqlContext.read.csv(root + "all_scores_reduced.scores",header=True, mode="DROPMALFORMED",sep="\t")

genenames = file.select("V1").rdd.flatMap(lambda x: x).collect()
alldf = file.collect()


allgenesvdsK=[]
with hadoop_read(root + 'all_gene_names.txt') as f:
    for line in f:
        allgenesvdsK=f.read().splitlines()
        

dictout = {}
for l in list(allgenesvdsK):
    if l in genenames:
        r = alldf[genenames.index(l)]
        ind=[k for k,i  in enumerate(r) if i=="1"]
        if len(ind) > 0:
            newdict = {l:[file.columns[i] for i in ind]}
        else:
            newdict = {l:[]}
    else:
        newdict = {l:[]}
    dictout.update(newdict)

Export total number genome-wide URVs per sample to use for adjustment in the model. _This output is not shared since it includes individual-level information_

In [28]:
## vds: input vds
## root: where to save the output
def export_scores_all(vds,root):
    (vds
    .export_samples(root + '/scores/URV_all.tsv','ID=s,COUNT=gs.filter(g => va.freq.URV).map(g => g.nNonRefAlleles).sum().toInt'))

    (vds
    .export_samples(root + '/scores/AF001_all.tsv','ID=s,COUNT=gs.filter(g => va.freq.AF001).map(g => g.nNonRefAlleles).sum().toInt'))


Export total number of clinvar and lethal variants. Export per-sample and also per-genotype, so we can count the average number per individual. _This output is not shared since it includes individual-level information_

In [28]:
def export_clinvar_lethal(vds,root):
    (vds
    .export_samples(root + '/scores/sc-clinvar-AF001-all.tsv','ID=s,COUNT=gs.filter(g => va.freq.AF001 && va.clinvar).map(g => g.nNonRefAlleles).sum().toInt'))

    (vds
    .filter_variants_expr('va.freq.AF001 && va.clinvar', keep=True)
    .export_genotypes(root + '/scores/va-clinvar.tsv','s, v, g.nNonRefAlleles'))

    (vds
    .export_samples(root + '/scores/sc-lethal-AF001-all.tsv','ID=s,COUNT=gs.filter(g => va.freq.AF001 && va.lethal).map(g => g.nNonRefAlleles).sum().toInt'))

    (vds
    .filter_variants_expr('va.freq.AF001 && va.lethal', keep=True)
    .export_genotypes(root + '/scores/va-lethal.tsv','s, v, g.nNonRefAlleles'))


We export a large .tsv file where the first column is the sample name, the second column is the gene name and the third colum is the number of alleles. This matrix will be used in the burden-test analysis. The file is then post-processed in R to create a matrix of samples x genes. _This output is not shared since it includes individual-level information_

In [1]:
## vds: input vds
## root: where to save the output

def export_sample_by_gene(vds,root):
    linreg_kt, sample_kt = (vds
    .filter_variants_expr('va.freq.AF001 && va.geneann.LOF == 1')
    .annotate_samples_expr('sa.pheno = pcoin(0.5)')
    .linreg_burden(key_name='gene',
                           variant_keys='va.geneann.gene',
                           single_key=True,
                           agg_expr='gs.map(g => g.gt).sum()',
                           y='sa.pheno',
                           covariates=[]))
    (sample_kt.export(root + "/genes_by_samples_AF001_LOF.tsv"))
    
    linreg_kt, sample_kt = (vds
    .filter_variants_expr('va.freq.AF001 && va.geneann.LOFdamaging == 1')
    .annotate_variants_expr('va.id=va.geneann.gene + "_" + v')
    .annotate_samples_expr('sa.pheno = pcoin(0.5)')
    .linreg_burden(key_name='gene',
                           variant_keys='va.id',
                           single_key=True,
                           agg_expr='gs.map(g => g.gt).sum()',
                           y='sa.pheno',
                           covariates=[]))
    (sample_kt.export(root + "/genes_by_samples_AF001_LOFdamaging.tsv.gz"))
    

Here we export the scores for all the gene-sets. 
We  considered 6 different classes of variants: Loss of function, loss of function + damaging, damaging (1 algorithm), damaging (3 algorithms), damaging (7 algorithms), damaging (as defined by badness score >= 2) synonymous.

We are exporting different scores:
- All heterozygous and homozygous mutations (we also export genotype-level data for figure 1)
- Only homozygous mutations
- Only indels
- Only SNPs
- No C->T or G->A
- Only high-quality variants as defined in step1 of this analysis and only regions well covered by all captures kit

_This output is not shared since it includes individual-level information_

In [2]:
def export_geneset_by_sample(vds,filter_expr,what_sum,out_tsv,genescores_dict):
    vdsT1=(vds
        .annotate_global('global.staged',genescores_dict,TDict(TString(),TArray(TString()))))
    vdsT2=(vdsT1.
         annotate_variants_expr('va.geneSets = global.staged[va.geneann.gene]'))

    linreg_kt, sample_kt = (vdsT2
        .filter_variants_expr(filter_expr, keep=True)
        .annotate_samples_expr('sa.pheno = pcoin(0.5)')
        .linreg_burden(key_name='geneSets',
                           variant_keys='va.geneSets',
                           single_key=False,
                           agg_expr='%s' % (what_sum),
                           y='sa.pheno',
                           covariates=[]))
    sample_kt.export(out_tsv)

    
def gen_scores_ext(vds,genescores_dict,root_dir,DBS=False):
    freqfilter=['URV','AF001']
    dfilter=['LOF','LOFdamaging','dama1','dama3','dama7','damaMPC','synonymous']
    
    
    high_qual_variants = (hc.import_table(root_dir + '/high_quality_for_age_analysis.tsv',no_header=True, impute=True)
        .annotate('variant = f0')
        .key_by('variant'))
    
    alllist_ext = list(itertools.product(freqfilter, dfilter))
 
    for x in alllist_ext:
        print(x)
        filterEX='va.freq.%s && va.geneann.%s == 1' % (x[0],x[1])
        export_geneset_by_sample(vds=vds,
                                 filter_expr=filterEX,
                                 what_sum='gs.map(g => g.nNonRefAlleles).sum()',
                                 out_tsv=root_dir + '/scores/sc-%s-%s-ALL.tsv' % (x[0],x[1]),
                                 genescores_dict=genescores_dict)

        filterEX='va.freq.%s && va.geneann.%s == 1' % (x[0],x[1])
        export_geneset_by_sample(vds=vds,
                                 filter_expr=filterEX,
                                 what_sum='gs.filter(g => g.isHomVar).count()',
                                 out_tsv=root_dir + '/scores/sc-%s-%s-HOM.tsv' % (x[0],x[1]),
                                 genescores_dict=genescores_dict)
         
        if DBS == False:
            filterEX= 'va.freq.%s && va.geneann.%s == 1' % (x[0],x[1])
            vdsT2 = (vds
                     .filter_variants_table(high_qual_intervals)
                     .filter_variants_table(high_qual_variants))
            export_geneset_by_sample(vds=vdsT2,
                                     filter_expr=filterEX,
                                     what_sum='gs.map(g => g.nNonRefAlleles).sum()',
                                     out_tsv=root_dir + '/scores/sc-%s-%s-HIQFORAGE.tsv' % (x[0],x[1]),
                                     genescores_dict=genescores_dict)  
            
            filterEX='v.altAllele.isSNP && va.freq.%s && va.geneann.%s == 1' % (x[0],x[1])
            export_geneset_by_sample(vds=vds,
                                     filter_expr=filterEX,
                                     what_sum='gs.map(g => g.nNonRefAlleles).sum()',
                                     out_tsv=root_dir + '/scores/sc-%s-%s-SNP.tsv' % (x[0],x[1]),
                                     genescores_dict=genescores_dict)

            filterEX='''
                        va.freq.%s && va.geneann.%s == 1 && 
                        v.altAllele.isSNP && ((v.altAllele.ref!="C" &&  v.altAllele.alt!="T") && 
                        (v.altAllele.ref!="G" && v.altAllele.alt!="A"))
                        ''' % (x[0],x[1])
            export_geneset_by_sample(vds=vds,
                                     filter_expr=filterEX,
                                     what_sum='gs.map(g => g.nNonRefAlleles).sum()',
                                     out_tsv=root_dir + '/scores/sc-%s-%s-SNPNOCTGA.tsv' % (x[0],x[1]),
                                     genescores_dict=genescores_dict)

            if x[1] in ['LOF','LOFdamaging','synonymous']:
                
                filterEX='v.altAllele.isIndel && va.freq.%s && va.geneann.%s == 1' % (x[0],x[1])
                export_geneset_by_sample(vds=vds,
                                         filter_expr=filterEX,
                                         what_sum='gs.map(g => g.nNonRefAlleles).sum()',
                                         out_tsv=root_dir + '/scores/sc-%s-%s-INDEL.tsv' % (x[0],x[1]),
                                         genescores_dict=genescores_dict)         
            
            
        if x[1] in ['synonymous','LOF']:
            
            vdsT1=(vds
                   .filter_variants_expr('va.freq.%s && va.geneann.%s == 1' % (x[0],x[1]), keep=True)
                   .annotate_global('global.staged',dictout,TDict(TString(),TArray(TString())))
                   .annotate_variants_expr('va.geneSets = global.staged[va.geneann.gene]'))

            (vdsT1
             .filter_variants_expr('va.geneSets.toSet().contains("pHI09")')
             .export_genotypes(root_dir + '/scores/va-pHI09-%s-%s.tsv' % (x[0],x[1]),'s, v, g.nNonRefAlleles'))

            (vdsT1
             .filter_variants_expr('va.geneSets.toSet().contains("ALL")')
             .export_genotypes(root_dir + '/scores/va-ALL-%s-%s.tsv' % (x[0],x[1]),'s, v, g.nNonRefAlleles'))

            (vdsT1
             .filter_variants_expr('va.geneSets.toSet().contains("pLO01")')
             .export_genotypes(root_dir + '/scores/va-pLO01-%s-%s.tsv' % (x[0],x[1]),'s, v, g.nNonRefAlleles'))


# PART 3: Linear or logistic association analysis between scores and phenotypes 

The function below tests the association between each score and the phenotypes, adjusting for covariates. Code is in R.

We first define the phenotype file, the covariates and the outcomes as follow:

In [ ]:
## The example below is for the swedish-schizoprenia cohort, similar approach is used for all
## the cohorts/ethnicities in the study

path <- root
phenotypdata <- read.csv(paste0(path,"phenotypes.csv"), stringsAsFactors=F)
covariates <- c("age","sex","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")
suffix <- c("sweden/NO_ADJ_SCZ")
pheno <- c("scz","BMI","SYST","DIAS","LNGD","LNGD_RAW")
type <- c("D","C","C","C","C","C") # C=continous,D=dichotomous
POP <- c("nfe") # If POP is in your phenotype file, this keeps only that POP    

In [ ]:
URV_score <- read.table(paste0(path,"scores/URV_all.tsv"), stringsAsFactor=F, header=T)
AF001_score <- read.table(paste0(path,"scores/AF001_all.tsv"), stringsAsFactor=F, header=T)
DOUBLEURV_score <- read.table(paste0(path,"scores/DOUBLEURV_all.tsv"), stringsAsFactor=F, header=T)

#### ALL SCORES AND VARIANTS ###
allscores <- list.files(paste0(path,"scores/"), pattern="^sc-*")

## We exclude these scores because only used in the sensitivity analysis the age-analysis ###
allscores <- allscores[grepl("LOF|LOFdamaging|dama1|dama3|dama7|synonymous",allscores) & !grepl("SNP|INDEL|SNPNOCTGA|HIQFORAGE|HOM",allscores)]


MATSCORE <- matrix(,ncol=length(allscores), nrow=nrow(URV_score))
GENSETL <- NULL
FREQL <- NULL
DFILTERL <- NULL
VARTYPE <- NULL
for (nscore in 1:length(allscores))
{
	score <- read.table(paste0(path,"scores/",allscores[nscore]), stringsAsFactor=F, header=T, sep="\t")

	ch <- strsplit(gsub(".tsv","",allscores[nscore]),"-")[[1]]
	GENSETL <- c(GENSETL,ch[2])
	FREQL <- c(FREQL,ch[3])

	if (length(ch)==5)
	{
		DFILTERL <- c(DFILTERL,paste0(ch[4],"HOM"))
	}else
	{
		DFILTERL <- c(DFILTERL,ch[4])
	}

	MATSCORE[,nscore] <- score$COUNT
}


## ADD SYNOSCORE (which will be at the end)
MATSCOREM <- data.frame(ID=URV_score$ID,MATSCORE,S_URV=URV_score[,2], S_AF001=AF001_score[,2], S_SING=SING_score[,2], S_DOUBLEURV=DOUBLEURV_score[,2],stringsAsFactors=F)

### MERGE PHENOTYPE WITH ALL THE SCORES, BUT THE SCORE WILL BE IN THE FRONT ###

mergeddata <- merge(MATSCOREM,phenotypdata,by.x="ID", by.y="ID")

### SELECT ONLY SPECIFIED POPS ####

mergeddata <- mergeddata[mergeddata$POP %in% POP,]

print(paste0("N before exclusions:",nrow(mergeddata)))

### EXCLUDE INDIVIDUALS WITH median(x) +/- 4 MADs number of SYNONYMOUS SINGLETONS ###
if (mad(mergeddata$S_URV) == 0 )
{
	mergeddata <- mergeddata[(mergeddata$S_URV < median(mergeddata$S_URV) + 4*mad(mergeddata$S_URV)) & (mergeddata$S_URV > median(mergeddata$S_URV) - 4*mad(mergeddata$S_URV)),]

	print(paste0("N after exclusions:",nrow(mergeddata)))
} else
{
	mergeddata <- mergeddata[(mergeddata$S_URV < median(mergeddata$S_URV) + 4*sd(mergeddata$S_URV)) & (mergeddata$S_URV > median(mergeddata$S_URV) - 4*sd(mergeddata$S_URV)),]

	print(paste0("N after exclusions:",nrow(mergeddata)))
}

phenotypdataX <- mergeddata[,colnames(mergeddata) %in% c(colnames(phenotypdata),"S_URV","S_AF001","S_SING","S_DOUBLEURV")]


lhciC <- function(x)
{
  
coSYNO <- coef(x)[grepl("^S_",names(coef(x)))]
seSYNO <- coef(summary(x))[grepl("^S_",rownames(coef(summary(x)))),2]
pSYNO <- coef(summary(x))[grepl("^S_",rownames(coef(summary(x)))),4]

namex <- names(coef(x))[grepl("^X",names(coef(x)))]

if(is.na(coef(x)[grepl("^X",names(coef(x)))]))

{
	coX <- NA
	pX <- NA
	seX <- NA
	hiX <- NA
	liX <- NA
}
else
{
	coX <- coef(x)[grepl("^X",names(coef(x)))]
	pX <- coef(summary(x))[grepl("^X",rownames(coef(summary(x)))),4]
	seX <- coef(summary(x))[grepl("^X",rownames(coef(summary(x)))),2]
	hiX <- coX + 1.96 * seX
	liX <- coX - 1.96 * seX
}

if(x$family$family=="binomial")
{
	N <- table(x$y)
	TOTVAR <- c(sum(x$model[x$y==0,namex]),sum(x$model[x$y==1,namex]))
}

else
{
	N <- length(x$y)
	TOTVAR <- sum(x$model[,namex])
}

return(c(coSYNO,seSYNO,pSYNO,coX,pX,seX,hiX,liX,N,TOTVAR))
}



if ("C" %in% type)
{
	phenosC <- pheno[type=="C"]

	print(paste0("Running analysis on continuous phenotypes: ",paste(phenosC, collapse=",")))

	RESCOREC <- NULL
	RESVARC <- NULL
	RESVARCG <- NULL
	for (p in phenosC)
	{

		if ("COHORT" %in% covariates & nlevels(factor(mergeddata$COHORT[!is.na(mergeddata[[p]])]))==1)
		{covariatesnew <- covariates[!covariates %in% "COHORT"]} else {covariatesnew <- covariates}

		fbase <- as.formula(paste0(p,"~",paste(covariatesnew,collapse="+")," + S_AF001"))
		print(p)
		

		for (nscore in 1:ncol(MATSCORE))
		{

		    f <- as.formula(paste0(p,"~",paste(covariatesnew,collapse="+")," + S_",FREQL[nscore],"+ X",nscore))
		    mod <- glm(f, data=mergeddata)
		    csm <- coef(summary(mod))
		    RESCOREC <- rbind(RESCOREC,c(p,GENSETL[nscore],FREQL[nscore],DFILTERL[nscore],lhciC(mod)))
		}

		colnames(RESCOREC) <- c("pheno","geneset_name","freq_thsld","dfilter","beta_syno","se_syno","p_syno","beta_x","p_x","se_x","hi_x","li_x","N","TOTVAR")

	}

write.table(RESCOREC,file=paste0(path,suffix,"_CONT_BURDEN.txt"), quote=F, row.names=F, sep="\t")
print(p)

}



if ("D" %in% type)
{
	phenosD <- pheno[type=="D"]

	print(paste0("Running analysis on dichotomus phenotypes: ",paste(phenosD, collapse=",")))

	RESCORED <- NULL
	RESVARD <- NULL
	RESVARDG <- NULL
	for (p in phenosD)
	{

		
		if ("COHORT" %in% covariates & nlevels(factor(mergeddata$COHORT[!is.na(mergeddata[[p]])]))==1) {covariatesnew <- covariates[!covariates %in% "COHORT"]} else {covariatesnew <- covariates}

		fbase <- as.formula(paste0(p,"~",paste(covariatesnew,collapse="+")," + S_AF001"))
		print(p)

		for (nscore in 1:ncol(MATSCORE))
		{

		   	f <- as.formula(paste0(p,"~",paste(covariatesnew,collapse="+")," + S_",FREQL[nscore],"+ X",nscore))
		    mod <- glm(f, data=mergeddata,family=binomial())
		    csm <- coef(summary(mod))
		    RESCORED <- rbind(RESCORED,c(p,GENSETL[nscore],FREQL[nscore],DFILTERL[nscore],lhciC(mod)))
		}

		colnames(RESCORED) <- c("pheno","geneset_name","freq_thsld","dfilter","beta_syno","se_syno","p_syno","beta_x","p_x","se_x","hi_x","li_x","N_CNTL","N_CASES","TOTVAR_CNTL","TOTVAR_CASES")

	}

write.table(RESCORED,file=paste0(path,suffix,"_DICOT_BURDEN.txt"), quote=F, row.names=F, sep="\t")
print(p)
}



# PART 4: SKAT Burden-test for association between scores and phenotypes

We test the association between each score and the phenotypes using burden, SKAT or SKAT-O test. The impute is the same of the one used in the function above

In [ ]:
library(reshape2)
library(SKAT)


## LOAD THE TWO R MATRIX CONTAINING SAMPLE BY GENE INFORMATION ##
load(paste0(path,"genes_by_samples_AF001_LOF_matrix.Rdata"))
load(paste0(path,"genes_by_samples_AF001_LOF_dama_matrix.Rdata"))


## SYSNONYMOUS FOR ADJUSTMENT ##
URV_score <- read.table(paste0(path,"scores/URV_all.tsv"), stringsAsFactor=F, header=T)
AF001_score <- read.table(paste0(path,"scores/AF001_all.tsv"), stringsAsFactor=F, header=T)
DOUBLEURV_score <- read.table(paste0(path,"scores/DOUBLEURV_all.tsv"), stringsAsFactor=F, header=T)


## This folder contains all the gene-sets to be tested, 
## each file is simply a .txt file containing the gene names for that specific geneset 

allgenefiles <- list.files("dir_with_genelists_for_burden")


## ADD SYNOSCORE (which will be at the end)
MATSCOREM <- data.frame(ID=URV_score$ID,MATSCORE,S_URV=URV_score[,2], S_AF001=AF001_score[,2], S_SING=SING_score[,2], S_DOUBLEURV=DOUBLEURV_score[,2],stringsAsFactors=F)

### MERGE PHENOTYPE WITH ALL THE SCORES, BUT THE SCORE WILL BE IN THE FRONT ###

mergeddata <- merge(MATSCOREM,phenotypdata,by.x="ID", by.y="ID")

### SELECT ONLY SPECIFIED POPS ####

mergeddata <- mergeddata[mergeddata$POP %in% POP,]

print(paste0("N before exclusions:",nrow(mergeddata)))

### EXCLUDE INDIVIDUALS WITH median(x) +/- 4 MADs number of SYNONYMOUS SINGLETONS ###
if (mad(mergeddata$S_URVEXACV2EHR) == 0 )
{
	mergeddata <- mergeddata[(mergeddata$S_URVEXACV2EHR < median(mergeddata$S_URVEXACV2EHR) + 4*mad(mergeddata$S_URVEXACV2EHR)) & (mergeddata$S_URVEXACV2EHR > median(mergeddata$S_URVEXACV2EHR) - 4*mad(mergeddata$S_URVEXACV2EHR)),]

	print(paste0("N after exclusions:",nrow(mergeddata)))
} else
{
	mergeddata <- mergeddata[(mergeddata$S_URVEXACV2EHR < median(mergeddata$S_URVEXACV2EHR) + 4*sd(mergeddata$S_URVEXACV2EHR)) & (mergeddata$S_URVEXACV2EHR > median(mergeddata$S_URVEXACV2EHR) - 4*sd(mergeddata$S_URVEXACV2EHR)),]

	print(paste0("N after exclusions:",nrow(mergeddata)))
}

phenotypdataX <- mergeddata[,colnames(mergeddata) %in% c(colnames(phenotypdata),"S_URVEXACV2EHR","S_AF001","S_SING","S_DOUBLEEXACV2EHR","S_AF01")]



lhciC <- function(x)
{
  
coSYNO <- coef(x)[grepl("^S_",names(coef(x)))]
seSYNO <- coef(summary(x))[grepl("^S_",rownames(coef(summary(x)))),2]
pSYNO <- coef(summary(x))[grepl("^S_",rownames(coef(summary(x)))),4]

namex <- names(coef(x))[grepl("^X",names(coef(x)))]

if(is.na(coef(x)[grepl("^X",names(coef(x)))]))

{
	coX <- NA
	pX <- NA
	seX <- NA
	hiX <- NA
	liX <- NA
}
else
{
	coX <- coef(x)[grepl("^X",names(coef(x)))]
	pX <- coef(summary(x))[grepl("^X",rownames(coef(summary(x)))),4]
	seX <- coef(summary(x))[grepl("^X",rownames(coef(summary(x)))),2]
	hiX <- coX + 1.96 * seX
	liX <- coX - 1.96 * seX
}

if(x$family$family=="binomial")
{
	N <- table(x$y)
	TOTVAR <- c(sum(x$model[x$y==0,namex]),sum(x$model[x$y==1,namex]))
}

else
{
	N <- length(x$y)
	TOTVAR <- sum(x$model[,namex])
}

return(c(coX,pX,seX,N,TOTVAR))
}


##### CONTINUOUS OUTCOME ######

if ("C" %in% type)
{
	phenosC <- pheno[type=="C"]

	print(paste0("Running analysis on continuous phenotypes: ",paste(phenosC, collapse=",")))


	RESVARCG <- NULL
	for (p in phenosC)
	{

		if ("COHORT" %in% covariates & nlevels(factor(mergeddata$COHORT[!is.na(mergeddata[[p]])]))==1)
		{covariatesnew <- covariates[!covariates %in% "COHORT"]} else {covariatesnew <- covariates}

		fbase <- as.formula(paste0(p,"~",paste(covariatesnew,collapse="+")," + S_AF001"))
		print(p)
		


		phenotypdataS <- phenotypdataX[complete.cases(phenotypdataX[,all.vars(fbase)]),]

		if ("COHORT" %in% covariates)
				{phenotypdataS$COHORT <- factor(phenotypdataS$COHORT)}

		exac_gbS <- gene_sample_AF001_LOF_mat[gene_sample_AF001_LOF_mat$Sample %in% phenotypdataS$ID,]
		exac_gbSdama <- gene_sample_AF001_LOF_dama_mat[gene_sample_AF001_LOF_dama_mat$Sample %in% phenotypdataS$ID,]

		

		for (ngene in 1:length(allgenefiles))
		{

		if (file.size(paste0("dir_with_genelists_for_burden",allgenefiles[ngene])) >0 )
	   		{
				dgene <- read.table(paste0("dir_with_genelists_for_burden",allgenefiles[ngene]), stringsAsFactor=F, header=F, sep="\t")

				RESTEMP1 <- NULL
				RESTEMP2 <- NULL

				exac_gbT <- exac_gbS[,colnames(exac_gbS) %in% dgene$V1,drop=FALSE]
				exac_gbTdama <- exac_gbSdama[,colnames(exac_gbSdama) %in% dgene$V1,drop=FALSE]

				if (nrow(exac_gbT) > 3 & ncol(exac_gbT[,colSums(exac_gbT)!=0,drop=FALSE]) > 1 )
			   	{

			   		######## LOF ############
			   		#########################

			   	
					genosm <- exac_gbT[order(match(exac_gbS$Sample,phenotypdataS$ID)),colSums(exac_gbT)!=0,drop=FALSE]
					genosm <- data.matrix(genosm)
					XBURDEN <- rowSums(genosm)
					fbaseused <- update(fbase, ~ . + XBURDEN)

					### Linear association (Burden) ###
		    		mod <- glm(fbaseused, data=phenotypdataS)
		   			csm <- coef(summary(mod))
		  			reslin <- lhciC(mod)


		  			#### SKAT-O TEST #####
					obj <- SKAT_Null_Model(fbase,out_type="C",n.Resampling=0, type.Resampling="bootstrap", Adjustment=FALSE, data=phenotypdataS)
					skmod <- tryCatch({SKAT(genosm,obj,kernel="linear",method="optimal.adj", impute.method="bestguess")}, error = function(err) {"SKAT_ERROR"})
					if (skmod[1]!="SKAT_ERROR")
					{
						if(!is.null(skmod$param$rho_est))
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- skmod$param$rho_est[1]
						pvalSKAT <- skmod$param$p.val.each[1]}
						else
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- 99
						pvalSKAT <- skmod$p.value}
					}else
					{
						pvalSKATO <- NA
						rhoSKATO <- NA
						pvalSKAT <- NA
					}


					RESTEMP1 <- c(p,gsub(".txt","",allgenefiles[ngene]),"AF001","LOF",reslin,pvalSKAT,pvalSKATO,rhoSKATO,ncol(genosm))


					###### DAMA + LOF ######
					########################


					genosm <- exac_gbTdama[order(match(exac_gbSdama$Sample,phenotypdataS$ID)),colSums(exac_gbTdama)!=0,drop=FALSE]
					genosm <- data.matrix(genosm)
					XBURDEN <- rowSums(genosm)
					fbaseused <- update(fbase, ~ . + XBURDEN)

					### Linear association (Burden) ###
		    		mod <- glm(fbaseused, data=phenotypdataS)
		   			csm <- coef(summary(mod))
		  			reslin <- lhciC(mod)


		  			#### SKAT-O TEST #####
					obj <- SKAT_Null_Model(fbase,out_type="C",n.Resampling=0, type.Resampling="bootstrap", Adjustment=FALSE, data=phenotypdataS)
					skmod <- tryCatch({SKAT(genosm,obj,kernel="linear",method="optimal.adj", impute.method="bestguess")}, error = function(err) {"SKAT_ERROR"})
					if (skmod[1]!="SKAT_ERROR")
					{
						if(!is.null(skmod$param$rho_est))
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- skmod$param$rho_est[1]
						pvalSKAT <- skmod$param$p.val.each[1]}
						else
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- 99
						pvalSKAT <- skmod$p.value}
					}else
					{
						pvalSKATO <- NA
						rhoSKATO <- NA
						pvalSKAT <- NA
					}

					RESTEMP2 <- c(p,gsub(".txt","",allgenefiles[ngene]),"AF001","LOFdamaging",reslin,pvalSKAT,pvalSKATO,rhoSKATO,ncol(genosm))

					RESVARCG <- rbind(RESVARCG,rbind(RESTEMP1,RESTEMP2))
				}

				
			}
		}			
		colnames(RESVARCG) <- c("pheno","geneset_name","freq_thsld","dfilter","co_burden","p_burden","se_burden","N","TOTVAR","p_x","SKATO_PVAL","rho_SKATO","nmarker")
	}

write.table(RESVARCG,file=paste0(path,suffix,"_CONT_SKAT.txt"), quote=F, row.names=F, sep="\t")
print(p)

}


##### DICHOTOMOUS OUTCOME ######

if ("D" %in% type)
{
	phenosD <- pheno[type=="D"]

	print(paste0("Running analysis on dichotomus phenotypes: ",paste(phenosD, collapse=",")))


	RESVARDG <- NULL
	for (p in phenosD)
	{

		
	  if ("COHORT" %in% covariates & nlevels(factor(mergeddata$COHORT[!is.na(mergeddata[[p]])]))==1)
		{covariatesnew <- covariates[!covariates %in% "COHORT"]} else {covariatesnew <- covariates}

		fbase <- as.formula(paste0(p,"~",paste(covariatesnew,collapse="+")," + S_AF001"))
		print(p)
		


		phenotypdataS <- phenotypdataX[complete.cases(phenotypdataX[,all.vars(fbase)]),]

		if ("COHORT" %in% covariates)
			{
				phenotypdataS$COHORT <- factor(phenotypdataS$COHORT)
				dep <- which(table(phenotypdataS$COHORT) < 2)	
				if (length(dep)>0)
				{
					levels(phenotypdataS$COHORT)[levels(phenotypdataS$COHORT)==names(dep)] <- names(table(phenotypdataS$COHORT))[1]
				}

			}

		exac_gbS <- gene_sample_AF001_LOF_mat[gene_sample_AF001_LOF_mat$Sample %in% phenotypdataS$ID,]
		exac_gbSdama <- gene_sample_AF001_LOF_dama_mat[gene_sample_AF001_LOF_dama_mat$Sample %in% phenotypdataS$ID,]



		for (ngene in 1:length(allgenefiles))
		{

		if (file.size(paste0("dir_with_genelists_for_burden",allgenefiles[ngene])) >0 )
	   		{
				dgene <- read.table(paste0("dir_with_genelists_for_burden",allgenefiles[ngene]), stringsAsFactor=F, header=F, sep="\t")


				RESTEMP1 <- NULL
				RESTEMP2 <- NULL

				exac_gbT <- exac_gbS[,colnames(exac_gbS) %in% dgene$V1,drop=FALSE]
				exac_gbTdama <- exac_gbSdama[,colnames(exac_gbSdama) %in% dgene$V1,drop=FALSE]

				if (nrow(exac_gbT) > 3 & ncol(exac_gbT[,colSums(exac_gbT)!=0,drop=FALSE]) > 1 )
			   	{

			   		######## LOF ############
			   		#########################

					genosm <- exac_gbT[order(match(exac_gbS$Sample,phenotypdataS$ID)),colSums(exac_gbT)!=0,drop=FALSE]
					genosm <- data.matrix(genosm)
					XBURDEN <- rowSums(genosm)
					fbaseused <- update(fbase, ~ . + XBURDEN)

					### Linear association (Burden) ###
		    		mod <- glm(fbaseused, data=phenotypdataS, family=binomial())
		   			csm <- coef(summary(mod))
		  			reslin <- lhciC(mod)


		  			#### SKAT-O TEST #####


					obj <- SKAT_Null_Model(fbase,out_type="D",n.Resampling=0, type.Resampling="bootstrap", Adjustment= FALSE, data=phenotypdataS)
					skmod <- tryCatch({SKATBinary(genosm,obj,kernel="linear",method="SKATO", impute.method="bestguess")}, error = function(err) {"SKAT_ERROR"})


					if (skmod[1]!="SKAT_ERROR")
					{
						if(!is.null(skmod$param$rho_est))
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- skmod$param$rho_est[1]
						pvalSKAT <- skmod$param$p.val.each[1]}
						else
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- 99
						pvalSKAT <- skmod$p.value}
					}
					else 
					{
						pvalSKATO <- NA
						rhoSKATO <- NA
						pvalSKAT <- NA
					}

				

					RESTEMP1 <- c(p,gsub(".txt","",allgenefiles[ngene]),"AF001","LOF",reslin,pvalSKAT,pvalSKATO,rhoSKATO,ncol(genosm))

					###### DAMA + LOF ######
					########################


					genosm <- exac_gbTdama[order(match(exac_gbSdama$Sample,phenotypdataS$ID)),colSums(exac_gbTdama)!=0,drop=FALSE]
					genosm <- data.matrix(genosm)
					XBURDEN <- rowSums(genosm)
					fbaseused <- update(fbase, ~ . + XBURDEN)

					### Linear association (Burden) ###
		    		mod <- glm(fbaseused, data=phenotypdataS, family=binomial())
		   			csm <- coef(summary(mod))
		  			reslin <- lhciC(mod)


		  			#### SKAT-O TEST #####
					obj <- SKAT_Null_Model(fbase,out_type="D",n.Resampling=0, type.Resampling="bootstrap", Adjustment=FALSE, data=phenotypdataS)
					skmod <- tryCatch({SKATBinary(genosm,obj,kernel="linear",method="SKATO", impute.method="bestguess")}, error = function(err) {"SKAT_ERROR"})
					if (skmod[1]!="SKAT_ERROR")
					{
						if(!is.null(skmod$param$rho_est))
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- skmod$param$rho_est[1]
						pvalSKAT <- skmod$param$p.val.each[1]}
						else
						{pvalSKATO <- skmod$p.value
						rhoSKATO <- 99
						pvalSKAT <- skmod$p.value}
					}
					else 
					{
						pvalSKATO <- NA
						rhoSKATO <- NA
						pvalSKAT <- NA
					}


					RESTEMP2 <- c(p,gsub(".txt","",allgenefiles[ngene]),"AF001","LOFdamaging",reslin,pvalSKAT,pvalSKATO,rhoSKATO,ncol(genosm))

					RESVARDG <- rbind(RESVARDG,rbind(RESTEMP1,RESTEMP2))
		
				}

				
			}
		}

		colnames(RESVARDG) <- c("pheno","geneset_name","freq_thsld","dfilter","co_burden","p_burden","se_burden","N_CNTL","N_CASES","TOTVAR_CNTL","TOTVAR_CASES","p_x","SKATO_PVAL","rho_SKATO","nmarker")
	}

write.table(RESVARDG,file=paste0(path,suffix,"_DICOT_SKAT.txt"), quote=F, row.names=F, sep="\t")

print(p)
}


# PART 5: meta-analysis of all results

In this part we meta-analyze the results, for each sub-study and ethnicity within the sub-studies. Code in R.

First we read in all the .txt files generated from the previous functions.

In [ ]:
library(meta)
library(metap)

#############################
### T2D-GENES/GoT2D/SIGMA ###
#############################


#### BURDEN ####

### Continuous outcomes ###
diabC_afrB <- "results_in_afr_CONT_BURDEN.txt"
diabC_amrB <- "results_in_amr_CONT_BURDEN.txt"
diabC_easB <- "results_in_eas_CONT_BURDEN.txt"
diabC_nfeB <- "results_in_nfe_CONT_BURDEN.txt"
diabC_finB <- "results_in_fin_CONT_BURDEN.txt"
diabC_sasB <- "results_in_sas_CONT_BURDEN.txt"

### Adjusted by BMI ###
diabC2_afrB <- "results_in_afr_bmi_adj_CONT_BURDEN.txt"
diabC2_amrB <- "results_in_amr_bmi_adj_CONT_BURDEN.txt"
diabC2_easB <- "results_in_eas_bmi_adj_CONT_BURDEN.txt"
diabC2_nfeB <- "results_in_nfe_bmi_adj_CONT_BURDEN.txt"
diabC2_finB <- "results_in_fin_bmi_adj_CONT_BURDEN.txt"
diabC2_sasB <- "results_in_sas_bmi_adj_CONT_BURDEN.txt"


### Not adjusted by age ###
diabC_afrB <- "results_in_afr_AGE_CONT_BURDEN.txt"
diabC_amrB <- "results_in_amr_AGE_CONT_BURDEN.txt"
diabC_easB <- "results_in_eas_AGE_CONT_BURDEN.txt"
diabC_nfeB <- "results_in_nfe_AGE_CONT_BURDEN.txt"
diabC_finB <- "results_in_fin_AGE_CONT_BURDEN.txt"
diabC_sasB <- "results_in_sas_AGE_CONT_BURDEN.txt"


### Not adjusted by t2d ###
diabD_afrB <- "results_in_afr_T2D_DICOT_BURDEN.txt"
diabD_amrB <- "results_in_amr_T2D_DICOT_BURDEN.txt"
diabD_easB <- "results_in_eas_T2D_DICOT_BURDEN.txt"
diabD_nfeB <- "results_in_nfe_T2D_DICOT_BURDEN.txt"
diabD_finB <- "results_in_fin_T2D_DICOT_BURDEN.txt"
diabD_sasB <- "results_in_sas_T2D_DICOT_BURDEN.txt"


#### SKAT ####

### Continuous outcomes ###
SdiabC_afrB <- "results_in_afr_CONT_SKAT.txt"
SdiabC_amrB <- "results_in_amr_CONT_SKAT.txt"
SdiabC_easB <- "results_in_eas_CONT_SKAT.txt"
SdiabC_nfeB <- "results_in_nfe_CONT_SKAT.txt"
SdiabC_finB <- "results_in_fin_CONT_SKAT.txt"
SdiabC_sasB <- "results_in_sas_CONT_SKAT.txt"

### Adjusted by BMI ###
SdiabC2_afrB <- "results_in_afr_bmi_adj_CONT_SKAT.txt"
SdiabC2_amrB <- "results_in_amr_bmi_adj_CONT_SKAT.txt"
SdiabC2_easB <- "results_in_eas_bmi_adj_CONT_SKAT.txt"
SdiabC2_nfeB <- "results_in_nfe_bmi_adj_CONT_SKAT.txt"
SdiabC2_finB <- "results_in_fin_bmi_adj_CONT_SKAT.txt"
SdiabC2_sasB <- "results_in_sas_bmi_adj_CONT_SKAT.txt"


### Not adjusted by age ###
SdiabC_afrB <- "results_in_afr_AGE_CONT_SKAT.txt"
SdiabC_amrB <- "results_in_amr_AGE_CONT_SKAT.txt"
SdiabC_easB <- "results_in_eas_AGE_CONT_SKAT.txt"
SdiabC_nfeB <- "results_in_nfe_AGE_CONT_SKAT.txt"
SdiabC_finB <- "results_in_fin_AGE_CONT_SKAT.txt"
SdiabC_sasB <- "results_in_sas_AGE_CONT_SKAT.txt"


### Not adjusted by t2d ###
SdiabD_afrB <- "results_in_afr_T2D_DICOT_SKAT.txt"
SdiabD_amrB <- "results_in_amr_T2D_DICOT_SKAT.txt"
SdiabD_easB <- "results_in_eas_T2D_DICOT_SKAT.txt"
SdiabD_nfeB <- "results_in_nfe_T2D_DICOT_SKAT.txt"
SdiabD_finB <- "results_in_fin_T2D_DICOT_SKAT.txt"
SdiabD_sasB <- "results_in_sas_T2D_DICOT_SKAT.txt"



###########################################
### Swedish schizophrenia case/control ####
###########################################

#### Burden ####

### Continuous outcomes ###
sweC_nfeB <- "results_in_nfe_ADJ_SCZ_CONT_BURDEN.txt"

### Dichotomous outcomes ###
sweD_nfeB <- "results_in_nfe_ADJ_SCZ_DICOT_BURDEN.txt"

### Not adjusted by schizophrenia, Continuous ###
sweC2_nfeB <- "results_in_nfe_NO_ADJ_SCZ_CONT_BURDEN.txt"

### Not adjusted by age ###
sweC3_nfeB <- "results_in_nfe_AGE_SCZ_ALL_CONT_BURDEN.txt"

### Not adjusted by schizophrenia, Dichotomous ###
sweD2_nfeB <- "results_in_nfe_NO_ADJ_SCZ_DICOT_BURDEN.txt"



#### SKAT ####

### Continuous outcomes ###
SsweC_nfeB <- "results_in_nfe_ADJ_SCZ_CONT_SKAT.txt"

### Dichotomous outcomes ###
SsweD_nfeB <- "results_in_nfe_ADJ_SCZ_DICOT_SKAT.txt"

### Not adjusted by schizophrenia, Continuous ###
SsweC2_nfeB <- "results_in_nfe_NO_ADJ_SCZ_CONT_SKAT.txt"

### Not adjusted by age ###
SsweC3_nfeB <- "results_in_nfe_AGE_SCZ_ALL_CONT_SKAT.txt"

### Not adjusted by schizophrenia, Dichotomous ###
SsweD2_nfeB <- "results_in_nfe_NO_ADJ_SCZ_DICOT_SKAT.txt"




#############################################
### North Finland Intellectual disability ###
#############################################

nfidD_finB <- "results_in_fin_DICOT_BURDEN.txt"

SnfidD_finB <- "results_in_fin_DICOT_SKAT.txt"


#####################
### IBD consortia ###
#####################


### BURDEN ###
ibdD_nfeB <- "results_in_nfe_DICOT_BURDEN.txt"
ibdD_finB <- "results_in_fin_DICOT_BURDEN.txt"
ibdD_askB <- "results_in_ask_DICOT_BURDEN.txt"

### SKAT ###

SibdD_nfeB <- "results_in_nfe_DICOT_SKAT.txt"
SibdD_finB <- "results_in_fin_DICOT_SKAT.txt"
SibdD_askB <- "results_in_ask_DICOT_SKAT.txt"



#############
### Migen ###
#############


### Continuous outcomes ###
migenC_afrB <- "results_in_afr_CONT_BURDEN.txt"
migenC_nfeB <- "results_in_nfe_CONT_BURDEN.txt"
migenC_sasB <- "results_in_sas_CONT_BURDEN.txt"


### Not adjusted by age ###
migenC2_nfeB <- "results_in_nfe_AGE_CONT_BURDEN.txt"
migenC2_afrB <- "results_in_afr_AGE_CONT_BURDEN.txt"
migenC2_sasB <- "results_in_sas_AGE_CONT_BURDEN.txt"

### Dichotomous outcomes ###
migenD_afrB <- "results_in_afr_DICOT_BURDEN.txt"
migenD_nfeB <- "results_in_nfe_DICOT_BURDEN.txt"
migenD_sasB <- "results_in_sas_DICOT_BURDEN.txt"

### Not adjusted by CHD ###
migenD2_afrB <- "results_in_afr_NO_CHD_DICOT_BURDEN.txt"
migenD2_nfeB <- "results_in_nfe_NO_CHD_DICOT_BURDEN.txt"
migenD2_sasB <- "results_in_sas_NO_CHD_DICOT_BURDEN.txt"

#### SKAT ####


### Continuous outcomes ###
SmigenC_afrB <- "results_in_afr_CONT_SKAT.txt"
SmigenC_nfeB <- "results_in_nfe_CONT_SKAT.txt"
SmigenC_sasB <- "results_in_sas_CONT_SKAT.txt"


### Not adjusted by age ###
SmigenC2_nfeB <- "results_in_nfe_AGE_CONT_SKAT.txt"
SmigenC2_afrB <- "results_in_afr_AGE_CONT_SKAT.txt"
SmigenC2_sasB <- "results_in_sas_AGE_CONT_SKAT.txt"

### Dichotomous outcomes ###
SmigenD_afrB <- "results_in_afr_DICOT_SKAT.txt"
SmigenD_nfeB <- "results_in_nfe_DICOT_SKAT.txt"
SmigenD_sasB <- "results_in_sas_DICOT_SKAT.txt"

### Not adjusted by CHD ###
SmigenD2_afrB <- "results_in_afr_NO_CHD_DICOT_SKAT.txt"
SmigenD2_nfeB <- "results_in_nfe_NO_CHD_DICOT_SKAT.txt"
SmigenD2_sasB <- "results_in_sas_NO_CHD_DICOT_SKAT.txt"



###############
### Finrisk ###
###############

#### BURDEN ####

### Continuous outcomes ###
finC_finB <- "results_in_fin_CONT_BURDEN.txt"

### Not adjusted by cohort, only for TG ###
finC2_finB <- "results_in_fin_NOCOHORT_CONT_BURDEN.txt"

### Adjusted by BMI ###
finC3_finB <- "results_in_fin_BMI_ADJ_CONT_BURDEN.txt"

### Not adjusted by age ###
finC4_finB <- "results_in_fin_AGE_ALL_CONT_BURDEN.txt"


#### SKAT ####

### Continuous outcomes ###
SfinC_finB <- "results_in_fin_CONT_SKAT.txt"

### Not adjusted by cohort, only for TG ###
SfinC2_finB <- "results_in_fin_NOCOHORT_CONT_SKAT.txt"

### Adjusted by BMI ###
SfinC3_finB <- "results_in_fin_BMI_ADJ_CONT_SKAT.txt"

### Not adjusted by age ###
SfinC4_finB <- "results_in_fin_AGE_ALL_CONT_SKAT.txt"




##########################
### Danish Blood Spot ####
##########################

### Continuous outcomes, Only controls ###
dbsC_nfeB <- "results_in_nfe_CNTL_CONT_BURDEN.txt"

### Only controls, not adjusted for age ###
dbsC2_nfeB <- "results_in_nfe_CNTL_AGE_CONT_BURDEN.txt"

### Age at onset within diseases ###
dbsCAUTISM_nfeB <- "results_in_nfe_AUTISM_CONT_BURDEN.txt"
dbsCADHD_nfeB <- "results_in_nfe_ADHD_CONT_BURDEN.txt"
dbsCSCZ_nfeB <- "results_in_nfe_SCZ_CONT_BURDEN.txt"
dbsCBP_nfeB <- "results_in_nfe_BP_CONT_BURDEN.txt"

### Dichotomous outcomes ###
dbsD_nfeB <- "results_in_nfe_DICOT_BURDEN.txt"

### Dichotomous outcomes, only controls ###
dbsD2_nfeB <-"results_in_nfe_CNTL_DICOT_BURDEN.txt"

### Sex-specific ###
dbsD3_nfeB <- "results_in_nfe_FEMALE_DICOT_BURDEN.txt"
dbsD4_nfeB <- "results_in_nfe_MALE_DICOT_BURDEN.txt"


#### SKAT ####

### Continuous outcomes, Only controls ###
SdbsC_nfeB <- "results_in_nfe_CNTL_CONT_SKAT.txt"

### Only controls, not adjusted for age ###
SdbsC2_nfeB <- "results_in_nfe_CNTL_AGE_CONT_SKAT.txt"

### Age at onset within diseases ###
SdbsCAUTISM_nfeB <- "results_in_nfe_AUTISM_CONT_SKAT.txt"
SdbsCADHD_nfeB <- "results_in_nfe_ADHD_CONT_SKAT.txt"
SdbsCSCZ_nfeB <- "results_in_nfe_SCZ_CONT_SKAT.txt"
SdbsCBP_nfeB <- "results_in_nfe_BP_CONT_SKAT.txt"

### Dichotomous outcomes ###
SdbsD_nfeB <- "results_in_nfe_DICOT_SKAT.txt"

### Dichotomous outcomes, only controls ###
SdbsD2_nfeB <-"results_in_nfe_CNTL_DICOT_SKAT.txt"

### Sex-specific ###
SdbsD3_nfeB <- "results_in_nfe_FEMALE_DICOT_SKAT.txt"
SdbsD4_nfeB <- "results_in_nfe_MALE_DICOT_SKAT.txt"




##############
### METSIM ###
##############

#### BURDEN #####

### Continuous outcomes ###
metC_finB <- "results_in_fin_CONT_BURDEN.txt"

### Not adjusted by age ###
metC2_finB <- "results_in_fin_AGE_CONT_BURDEN.txt"

### Adjusted by BMI ###
metC3_finB <- "results_in_fin_ADJ_BMI_CONT_BURDEN.txt"

#### SKAT #####

### Continuous outcomes ###
SmetC_finB <- "results_in_fin_CONT_SKAT.txt"

### Not adjusted by age ###
SmetC2_finB <- "results_in_fin_AGE_CONT_SKAT.txt"

### Adjusted by BMI ###
SmetC3_finB <- "results_in_fin_ADJ_BMI_CONT_SKAT.txt"


Function for continuous outcomes: ldl,hdl,tc,tg,sbp,dbp,height,bmi,glucose adjusted for bmi, insuline adjusted for bmi, years of education, age, birth weight adjusted for gestational age, age of diagnosis: schizophrenia, bipolar, autism, adhd.

The code is quite repetitive and we need to treat each phenotype separately because unfortunately the name of the phenotypes in each dataset is not the same

In [ ]:
METAC <- function(phenolist,phenolistS=NULL,ll)
{
  METARES <- NULL
  if (is.null(phenolistS))
  {
    for (k in 1:nrow(ll))
    {
      
      beta <- unlist(sapply(phenolist,function(x){x$beta_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se <- unlist(sapply(phenolist,function(x){x$se_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      N <- unlist(sapply(phenolist,function(x){x$N[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      NVAR <- unlist(sapply(phenolist,function(x){x$TOTVAR[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      beta_syno <- unlist(sapply(phenolist,function(x){x$beta_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se_syno <- unlist(sapply(phenolist,function(x){x$se_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      if (length(beta) > 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))
        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(as.numeric(N)),sum(as.numeric(NVAR)),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,NA,NA,NA,NA,NA,NA,NA,NA)

        METARES <- rbind(METARES,METAFIN)

      }
    }
  }
  else
  {
    for (k in 1:nrow(ll))
    {

      METAFIN <- NULL
      beta <- unlist(sapply(phenolist,function(x){x$beta_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se <- unlist(sapply(phenolist,function(x){x$se_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      N <- unlist(sapply(phenolist,function(x){x$N[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      NVAR <- unlist(sapply(phenolist,function(x){x$TOTVAR[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      beta_syno <- unlist(sapply(phenolist,function(x){x$beta_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se_syno <- unlist(sapply(phenolist,function(x){x$se_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      pSKAT <- unlist(sapply(phenolistS,function(x){x$p_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      pSKATO <- unlist(sapply(phenolistS,function(x){x$SKATO_PVAL[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      betaBURDEN <- unlist(sapply(phenolistS,function(x){x$co_burden[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      seBURDEN <- unlist(sapply(phenolistS,function(x){x$se_burden[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))


      SKATN <- unlist(sapply(phenolistS,function(x){x$nmarker[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      pSKAT[pSKAT==0] <- 1e-16
      pSKATO[pSKATO==0] <- 1e-16
      pSKAT[pSKAT>1] <- NA
      pSKATO[pSKATO>1] <- NA

      pSKAT <- pSKAT[!is.na(pSKAT)]
      pSKATO <- pSKATO[!is.na(pSKATO)]

      if (length(beta) > 0 & length(pSKAT) == 0 & length(pSKATO) == 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(as.numeric(N)),sum(as.numeric(NVAR)),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,NA,NA,NA,NA,NA,NA,NA,NA)

      }
      if (length(beta) > 0 & length(pSKAT) > 0 & length(pSKATO) > 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))
        modburden <- metagen(betaBURDEN,seBURDEN,studlab=seq(1:length(betaBURDEN)))


        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        if (length(pSKATO) > 1)
        {modskatO <- sumlog(pSKATO)$p}
        else
        {modskatO <- pSKATO}
        

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(as.numeric(N)),sum(as.numeric(NVAR)),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,modskat,modskatO,modburden$TE.fixed,modburden$seTE.fixed,modburden$pval.fixed,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),paste0(length(pSKATO),"/",length(phenolistS)))

      }

      if (length(beta) > 0 & length(pSKAT) > 0 & length(pSKATO) == 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))
        modburden <- metagen(betaBURDEN,seBURDEN,studlab=seq(1:length(betaBURDEN)))


        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(as.numeric(N)),sum(as.numeric(NVAR)),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,modskat,NA,modburden$TE.fixed,modburden$seTE.fixed,modburden$pval.fixed,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),NA)

      }

      if (length(beta) == 0 & length(pSKAT) > 0 & length(pSKATO) > 0)
      {

        modburden <- metagen(betaBURDEN,seBURDEN,studlab=seq(1:length(betaBURDEN)))

        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        if (length(pSKATO) > 1)
        {modskatO <- sumlog(pSKATO)$p}
        else
        {modskatO <- pSKATO}

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,modskat,modskatO,modburden$TE.fixed,modburden$seTE.fixed,modburden$pval.fixed,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),paste0(length(pSKATO),"/",length(phenolistS)))

      }

      if (length(beta) == 0 & length(pSKAT) > 0 & length(pSKATO) == 0)
      {

        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,modskat,NA,NA,NA,NA,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),NA)

      }

        METARES <- rbind(METARES,METAFIN)
    }
  }
colnames(METARES) <- c("geneset_name","freq_thsld","dfilter","N","NVAR","beta","se","lower_CI","upper_CI","pval","beta_syno","se_syno","lower_CI_syno","upper_CI_syno","pval_syno","SKAT_P","SKATO_P","BURDEN_BETA","BURDEN_SE","BURDEN_PVAL","SKAT_NVAR","N_studies_SKAT","N_studies_SKATO")
return(METARES)
}


#### DEFINE GENE-SETS TO INCLUDE ####
## We exclude only homozygous scores, scores including singletons and URV dubletons ###
geneset <- unique(c(finC_finB$geneset_name,SfinC_finB$geneset_name))
geneset <- geneset[!grepl("specific",geneset)]
freq_filter <- unique(c(finC_finB$freq_thsld,SfinC_finB$freq_thsld))
freq_filter <- freq_filter[!grepl("SING|DOUBLEURV",freq_filter)]
dfilter <- unique(c(finC_finB$dfilter,SfinC_finB$dfilter))
dfilter <- dfilter[!grepl("HOM$",dfilter)]


ll <- expand.grid(list(geneset,freq_filter,dfilter))


### LDL ###
print("LDL")

phenolist <- list(finC_finB[finC_finB$pheno == "LDL",],diabC_afrB[diabC_afrB$pheno=="LDLQQ",],diabC_amrB[diabC_amrB$pheno=="LDLQQ",],diabC_easB[diabC_easB$pheno=="LDLQQ",],diabC_nfeB[diabC_nfeB$pheno=="LDLQQ",],diabC_finB[diabC_finB$pheno=="LDLQQ",],diabC_sasB[diabC_sasB$pheno=="LDLQQ",],migenC_afrB[migenC_afrB$pheno=="norm_LDL_BASELINE_ADJ",],migenC_nfeB[migenC_nfeB$pheno=="norm_LDL_BASELINE_ADJ",],migenC_sasB[migenC_sasB$pheno=="norm_LDL_BASELINE_ADJ",],metC_finB[metC_finB$pheno=="LDLQQ",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "LDL",],SdiabC_afrB[SdiabC_afrB$pheno=="LDLQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="LDLQQ",],SdiabC_easB[SdiabC_easB$pheno=="LDLQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="LDLQQ",],SdiabC_finB[SdiabC_finB$pheno=="LDLQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="LDLQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_LDL_BASELINE_ADJ",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_LDL_BASELINE_ADJ",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_LDL_BASELINE_ADJ",],SmetC_finB[SmetC_finB$pheno=="LDLQQ",])

METALDL <- METAC(phenolist,phenolistS,ll)


### HDL ###
print("HDL")

phenolist <- list(finC_finB[finC_finB$pheno == "HDL",],diabC_afrB[diabC_afrB$pheno=="HDLQQ",],diabC_amrB[diabC_amrB$pheno=="HDLQQ",],diabC_easB[diabC_easB$pheno=="HDLQQ",],diabC_nfeB[diabC_nfeB$pheno=="HDLQQ",],diabC_finB[diabC_finB$pheno=="HDLQQ",],diabC_sasB[diabC_sasB$pheno=="HDLQQ",],migenC_afrB[migenC_afrB$pheno=="norm_HDL_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="norm_HDL_BASELINE",],migenC_sasB[migenC_sasB$pheno=="norm_HDL_BASELINE",],metC_finB[metC_finB$pheno=="HDLQQ",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "HDL",],SdiabC_afrB[SdiabC_afrB$pheno=="HDLQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="HDLQQ",],SdiabC_easB[SdiabC_easB$pheno=="HDLQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="HDLQQ",],SdiabC_finB[SdiabC_finB$pheno=="HDLQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="HDLQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_HDL_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_HDL_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_HDL_BASELINE",],SmetC_finB[SmetC_finB$pheno=="HDLQQ",])

METAHDL <- METAC(phenolist,phenolistS,ll)


### TC ###
print("TC")

phenolist <- list(finC_finB[finC_finB$pheno == "TC",],diabC_afrB[diabC_afrB$pheno=="CHOLQQ",],diabC_amrB[diabC_amrB$pheno=="CHOLQQ",],diabC_easB[diabC_easB$pheno=="CHOLQQ",],diabC_nfeB[diabC_nfeB$pheno=="CHOLQQ",],diabC_finB[diabC_finB$pheno=="CHOLQQ",],diabC_sasB[diabC_sasB$pheno=="CHOLQQ",],migenC_afrB[migenC_afrB$pheno=="norm_TOTCHOL_BASELINE_ADJ",],migenC_nfeB[migenC_nfeB$pheno=="norm_TOTCHOL_BASELINE_ADJ",],migenC_sasB[migenC_sasB$pheno=="norm_TOTCHOL_BASELINE_ADJ",],metC_finB[metC_finB$pheno=="TCQQ",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "TC",],SdiabC_afrB[SdiabC_afrB$pheno=="CHOLQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="CHOLQQ",],SdiabC_easB[SdiabC_easB$pheno=="CHOLQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="CHOLQQ",],SdiabC_finB[SdiabC_finB$pheno=="CHOLQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="CHOLQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_TOTCHOL_BASELINE_ADJ",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_TOTCHOL_BASELINE_ADJ",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_TOTCHOL_BASELINE_ADJ",],SmetC_finB[SmetC_finB$pheno=="TCQQ",])

METATC <- METAC(phenolist,phenolistS,ll)


### TG ###
print("TG")

phenolist <- list(finC2_finB[finC2_finB$pheno == "TG",],diabC_afrB[diabC_afrB$pheno=="TGQQ",],diabC_amrB[diabC_amrB$pheno=="TGQQ",],diabC_easB[diabC_easB$pheno=="TGQQ",],diabC_nfeB[diabC_nfeB$pheno=="TGQQ",],diabC_finB[diabC_finB$pheno=="TGQQ",],diabC_sasB[diabC_sasB$pheno=="TGQQ",],migenC_afrB[migenC_afrB$pheno=="norm_LOG_TRIG_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="norm_LOG_TRIG_BASELINE",],migenC_sasB[migenC_sasB$pheno=="norm_LOG_TRIG_BASELINE",],metC_finB[metC_finB$pheno=="TGQQ",])
phenolistS <- list(SfinC2_finB[SfinC2_finB$pheno == "TG",],SdiabC_afrB[SdiabC_afrB$pheno=="TGQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="TGQQ",],SdiabC_easB[SdiabC_easB$pheno=="TGQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="TGQQ",],SdiabC_finB[SdiabC_finB$pheno=="TGQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="TGQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_LOG_TRIG_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_LOG_TRIG_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_LOG_TRIG_BASELINE",],SmetC_finB[SmetC_finB$pheno=="TGQQ",])

METATG <- METAC(phenolist,phenolistS,ll)


### SBP ###
print("SBP")

phenolist <- list(finC_finB[finC_finB$pheno == "SBP",],sweC2_nfeB[sweC2_nfeB$pheno=="SYST",],diabC_afrB[diabC_afrB$pheno=="SBPQQ",],diabC_amrB[diabC_amrB$pheno=="SBPQQ",],diabC_easB[diabC_easB$pheno=="SBPQQ",],diabC_nfeB[diabC_nfeB$pheno=="SBPQQ",],diabC_finB[diabC_finB$pheno=="SBPQQ",],diabC_sasB[diabC_sasB$pheno=="SBPQQ",],migenC_afrB[migenC_afrB$pheno=="norm_SBP_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="norm_SBP_BASELINE",],migenC_sasB[migenC_sasB$pheno=="norm_SBP_BASELINE",],metC_finB[metC_finB$pheno=="SBPQQ",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "SBP",],SsweC2_nfeB[SsweC2_nfeB$pheno=="SYST",],SdiabC_afrB[SdiabC_afrB$pheno=="SBPQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="SBPQQ",],SdiabC_easB[SdiabC_easB$pheno=="SBPQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="SBPQQ",],SdiabC_finB[SdiabC_finB$pheno=="SBPQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="SBPQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_SBP_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_SBP_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_SBP_BASELINE",],SmetC_finB[SmetC_finB$pheno=="SBPQQ",])

METASBP <- METAC(phenolist,phenolistS,ll)


### DBP ###
print("DBP")

phenolist <- list(finC_finB[finC_finB$pheno == "DBP",],sweC2_nfeB[sweC2_nfeB$pheno=="DIAS",],diabC_afrB[diabC_afrB$pheno=="DBPQQ",],diabC_amrB[diabC_amrB$pheno=="DBPQQ",],diabC_easB[diabC_easB$pheno=="DBPQQ",],diabC_nfeB[diabC_nfeB$pheno=="DBPQQ",],diabC_finB[diabC_finB$pheno=="DBPQQ",],diabC_sasB[diabC_sasB$pheno=="DBPQQ",],migenC_afrB[migenC_afrB$pheno=="norm_DBP_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="norm_DBP_BASELINE",],migenC_sasB[migenC_sasB$pheno=="norm_DBP_BASELINE",],metC_finB[metC_finB$pheno=="DBP",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "DBP",],SsweC2_nfeB[SsweC2_nfeB$pheno=="DIAS",],SdiabC_afrB[SdiabC_afrB$pheno=="DBPQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="DBPQQ",],SdiabC_easB[SdiabC_easB$pheno=="DBPQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="DBPQQ",],SdiabC_finB[SdiabC_finB$pheno=="DBPQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="DBPQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_DBP_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_DBP_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_DBP_BASELINE",],SmetC_finB[SmetC_finB$pheno=="DBP",])

METADBP <- METAC(phenolist,phenolistS,ll)


## HEIGHT ##
print("HEIGHT")

phenolist <- list(finC_finB[finC_finB$pheno == "HEIGHT",],sweC2_nfeB[sweC2_nfeB$pheno=="LNGD",],migenC_afrB[migenC_afrB$pheno=="norm_HEIGHT_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="norm_HEIGHT_BASELINE",],migenC_sasB[migenC_sasB$pheno=="norm_HEIGHT_BASELINE",],metC_finB[metC_finB$pheno=="HEIGHTQQ",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "HEIGHT",],SsweC2_nfeB[SsweC2_nfeB$pheno=="LNGD",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_HEIGHT_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_HEIGHT_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_HEIGHT_BASELINE",],SmetC_finB[SmetC_finB$pheno=="HEIGHTQQ",])

METALNGD <- METAC(phenolist,phenolistS,ll)



### HEIGHT-RAW ###
print("HEIGHT-raw")

phenolist <- list(finC_finB[finC_finB$pheno == "HEIGHT_RAW",],sweC2_nfeB[sweC2_nfeB$pheno=="LNGD_RAW",],migenC_afrB[migenC_afrB$pheno=="HEIGHT_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="HEIGHT_BASELINE",],migenC_sasB[migenC_sasB$pheno=="HEIGHT_BASELINE",],metC_finB[metC_finB$pheno=="HEIGHT",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "HEIGHT_RAW",],SsweC2_nfeB[SsweC2_nfeB$pheno=="LNGD_RAW",],SmigenC_afrB[SmigenC_afrB$pheno=="HEIGHT_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="HEIGHT_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="HEIGHT_BASELINE",],SmetC_finB[SmetC_finB$pheno=="HEIGHT",])

METALNGDRAW <- METAC(phenolist,phenolistS,ll)


### BMI ###
print("BMI")

phenolist <- list(finC_finB[finC_finB$pheno == "BMI",],sweC2_nfeB[sweC2_nfeB$pheno=="BMI",],diabC_afrB[diabC_afrB$pheno=="BMIQQ",],diabC_amrB[diabC_amrB$pheno=="BMIQQ",],diabC_easB[diabC_easB$pheno=="BMIQQ",],diabC_nfeB[diabC_nfeB$pheno=="BMIQQ",],diabC_finB[diabC_finB$pheno=="BMIQQ",],diabC_sasB[diabC_sasB$pheno=="BMIQQ",],migenC_afrB[migenC_afrB$pheno=="norm_BMI_BASELINE",],migenC_nfeB[migenC_nfeB$pheno=="norm_BMI_BASELINE",],migenC_sasB[migenC_sasB$pheno=="norm_BMI_BASELINE",],metC_finB[metC_finB$pheno=="BMIQQ",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "BMI",],SsweC2_nfeB[SsweC2_nfeB$pheno=="BMI",],SdiabC_afrB[SdiabC_afrB$pheno=="BMIQQ",],SdiabC_amrB[SdiabC_amrB$pheno=="BMIQQ",],SdiabC_easB[SdiabC_easB$pheno=="BMIQQ",],SdiabC_nfeB[SdiabC_nfeB$pheno=="BMIQQ",],SdiabC_finB[SdiabC_finB$pheno=="BMIQQ",],SdiabC_sasB[SdiabC_sasB$pheno=="BMIQQ",],SmigenC_afrB[SmigenC_afrB$pheno=="norm_BMI_BASELINE",],SmigenC_nfeB[SmigenC_nfeB$pheno=="norm_BMI_BASELINE",],SmigenC_sasB[SmigenC_sasB$pheno=="norm_BMI_BASELINE",],SmetC_finB[SmetC_finB$pheno=="BMIQQ",])

METABMI <- METAC(phenolist,phenolistS,ll)


### GLUCOSEADJBMI ###
print("GLUCOSEADJBMI")

phenolist <- list(finC3_finB[finC3_finB$pheno == "GLUCOSE",],diabC2_afrB[diabC2_afrB$pheno=="FAST_GLUQQ",],diabC2_amrB[diabC2_amrB$pheno=="FAST_GLUQQ",],diabC2_easB[diabC2_easB$pheno=="FAST_GLUQQ",],diabC2_nfeB[diabC2_nfeB$pheno=="FAST_GLUQQ",],diabC2_finB[diabC2_finB$pheno=="FAST_GLUQQ",],diabC2_sasB[diabC2_sasB$pheno=="FAST_GLUQQ",],metC3_finB[metC3_finB$pheno=="GLUCOSEQQ",])
phenolistS <- list(SfinC3_finB[SfinC3_finB$pheno == "GLUCOSE",],SdiabC2_afrB[SdiabC2_afrB$pheno=="FAST_GLUQQ",],SdiabC2_amrB[SdiabC2_amrB$pheno=="FAST_GLUQQ",],SdiabC2_easB[SdiabC2_easB$pheno=="FAST_GLUQQ",],SdiabC2_nfeB[SdiabC2_nfeB$pheno=="FAST_GLUQQ",],SdiabC2_finB[SdiabC2_finB$pheno=="FAST_GLUQQ",],SdiabC2_sasB[SdiabC2_sasB$pheno=="FAST_GLUQQ",],SmetC3_finB[SmetC3_finB$pheno=="GLUCOSEQQ",])

METAADJGLUCOSE <- METAC(phenolist,phenolistS,ll)


### INSULINEADJBMI ###
print("INSADJBMI")

phenolist <- list(finC3_finB[finC3_finB$pheno == "INS",],diabC2_afrB[diabC2_afrB$pheno=="FAST_INSQQ",],diabC2_amrB[diabC2_amrB$pheno=="FAST_INSQQ",],diabC2_easB[diabC2_easB$pheno=="FAST_INSQQ",],diabC2_nfeB[diabC2_nfeB$pheno=="FAST_INSQQ",],diabC2_finB[diabC2_finB$pheno=="FAST_INSQQ",],diabC2_sasB[diabC2_sasB$pheno=="FAST_INSQQ",],metC3_finB[metC3_finB$pheno=="INSQQ",])
phenolistS <- list(SfinC3_finB[SfinC3_finB$pheno == "INS",],SdiabC2_afrB[SdiabC2_afrB$pheno=="FAST_INSQQ",],SdiabC2_amrB[SdiabC2_amrB$pheno=="FAST_INSQQ",],SdiabC2_easB[SdiabC2_easB$pheno=="FAST_INSQQ",],SdiabC2_nfeB[SdiabC2_nfeB$pheno=="FAST_INSQQ",],SdiabC2_finB[SdiabC2_finB$pheno=="FAST_INSQQ",],SdiabC2_sasB[SdiabC2_sasB$pheno=="FAST_INSQQ",],SmetC3_finB[SmetC3_finB$pheno=="INSQQ",])

METAADJINS <- METAC(phenolist,phenolistS,ll)


### EDUCATION ###
print("EDU")

phenolist <- list(finC_finB[finC_finB$pheno == "yearsofedu",],sweC_nfeB[sweC_nfeB$pheno=="yearsofedu",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "yearsofedu",],SsweC_nfeB[SsweC_nfeB$pheno=="yearsofedu",])

METAEDU <- METAC(phenolist,phenolistS,ll)


### EDUCATION - RAW ###
print("EDU-raw")

phenolist <- list(finC_finB[finC_finB$pheno == "yearsofedu_raw",],sweC_nfeB[sweC_nfeB$pheno=="yearsofedu_raw",])
phenolistS <- list(SfinC_finB[SfinC_finB$pheno == "yearsofedu_raw",],SsweC_nfeB[SsweC_nfeB$pheno=="yearsofedu_raw",])

METAEDURAW <- METAC(phenolist,phenolistS,ll)


### AGE ###
print("AGE")

phenolist <- list(finC4_finB[finC4_finB$pheno == "ageqq",],diabC3_afrB[diabC3_afrB$pheno=="ageqq",],diabC3_easB[diabC3_easB$pheno=="ageqq",],diabC3_nfeB[diabC3_nfeB$pheno=="ageqq",],diabC3_finB[diabC3_finB$pheno=="ageqq",],diabC3_sasB[diabC3_sasB$pheno=="ageqq",],sweC3_nfeB[sweC3_nfeB$pheno=="ageqq",],diabC3_amrB[diabC3_amrB$pheno=="ageqq",],migenC2_afrB[migenC2_afrB$pheno=="ageqq",],migenC2_nfeB[migenC2_nfeB$pheno=="ageqq",],migenC2_sasB[migenC2_sasB$pheno=="ageqq",],metC2_finB[metC2_finB$pheno=="AGEQQ",])
phenolistS <- list(SfinC4_finB[SfinC4_finB$pheno == "ageqq",],SdiabC3_afrB[SdiabC3_afrB$pheno=="ageqq",],SdiabC3_easB[SdiabC3_easB$pheno=="ageqq",],SdiabC3_nfeB[SdiabC3_nfeB$pheno=="ageqq",],SdiabC3_finB[SdiabC3_finB$pheno=="ageqq",],SdiabC3_sasB[SdiabC3_sasB$pheno=="ageqq",],SsweC3_nfeB[SsweC3_nfeB$pheno=="ageqq",],SdiabC3_amrB[SdiabC3_amrB$pheno=="ageqq",],SmigenC2_afrB[SmigenC2_afrB$pheno=="ageqq",],SmigenC2_nfeB[SmigenC2_nfeB$pheno=="ageqq",],SmigenC2_sasB[SmigenC2_sasB$pheno=="ageqq",],SmetC2_finB[SmetC2_finB$pheno=="AGEQQ",])

METAAGE <- METAC(phenolist,phenolistS,ll)


### AGE - RAW ###
phenolist <- list(finC4_finB[finC4_finB$pheno == "AGE",],diabC3_afrB[diabC3_afrB$pheno=="AGE",],diabC3_easB[diabC3_easB$pheno=="AGE",],diabC3_nfeB[diabC3_nfeB$pheno=="AGE",],diabC3_finB[diabC3_finB$pheno=="AGE",],diabC3_sasB[diabC3_sasB$pheno=="AGE",],sweC3_nfeB[sweC3_nfeB$pheno=="age",],diabC3_amrB[diabC3_amrB$pheno=="AGE",],migenC2_afrB[migenC2_afrB$pheno=="AGE",],migenC2_nfeB[migenC2_nfeB$pheno=="AGE",],migenC2_sasB[migenC2_sasB$pheno=="AGE",],metC2_finB[metC2_finB$pheno=="AGE",])
phenolistS <- list(SfinC4_finB[SfinC4_finB$pheno == "AGE",],SdiabC3_afrB[SdiabC3_afrB$pheno=="AGE",],SdiabC3_easB[SdiabC3_easB$pheno=="AGE",],SdiabC3_nfeB[SdiabC3_nfeB$pheno=="AGE",],SdiabC3_finB[SdiabC3_finB$pheno=="AGE",],SdiabC3_sasB[SdiabC3_sasB$pheno=="AGE",],SsweC3_nfeB[SsweC3_nfeB$pheno=="age",],SdiabC3_amrB[SdiabC3_amrB$pheno=="AGE",],SmigenC2_afrB[SmigenC2_afrB$pheno=="AGE",],SmigenC2_nfeB[SmigenC2_nfeB$pheno=="AGE",],SmigenC2_sasB[SmigenC2_sasB$pheno=="AGE",],SmetC2_finB[SmetC2_finB$pheno=="AGE",])

METAAGERAW <- METAC(phenolist,phenolistS,ll)


### BIRTH WEIGHT ADJ BMI ###
print("BWADJGAGE")

phenolist <- list(dbsC_nfeB[dbsC_nfeB$pheno=="BWGHT_ADJGE",])
phenolistS <- list(SdbsC_nfeB[SdbsC_nfeB$pheno=="BWGHT_ADJGE",])

METABWADJBMI <- METAC(phenolist,phenolistS,ll)


### SCZ AGE ###
phenolist <- list(dbsCSCZ_nfeB[dbsCSCZ_nfeB$pheno=="AGESCZ",])
phenolistS <- list(SdbsCSCZ_nfeB[SdbsCSCZ_nfeB$pheno=="AGESCZ",])

METASCZAGE <- METAC(phenolist,NULL,ll)


### BP AGE ###
phenolist <- list(dbsCBP_nfeB[dbsCBP_nfeB$pheno=="AGEBP",])
phenolistS <- list(SdbsCBP_nfeB[SdbsCBP_nfeB$pheno=="AGEBP",])

METABPAGE <- METAC(phenolist,NULL,ll)


### AUTISM AGE ###
phenolist <- list(dbsCAUTISM_nfeB[dbsCAUTISM_nfeB$pheno=="AGEAUTISM",])
phenolistS <- list(SdbsCAUTISM_nfeB[SdbsCAUTISM_nfeB$pheno=="AGEAUTISM",])

METAAUTISMAGE <- METAC(phenolist,NULL,ll)

### ADHD AGE ###
phenolist <- list(dbsCADHD_nfeB[dbsCADHD_nfeB$pheno=="AGEADHDD",])
phenolistS <- list(SdbsCADHD_nfeB[SdbsCADHD_nfeB$pheno=="AGEADHDD",])

METAADHDAGE <- METAC(phenolist,NULL,ll)



Function for dichotomous outcomes: t2d, ibd, uc, cd, early onset mi, chd, id, scz, bp, autism, adhd. Dor scz, bp, autism, adhd we considered male and female separately, as well as having subsets considering no overlap with with other relevant co-morbidities

In [ ]:
METAD <- function(phenolist,phenolistS=NULL,ll)
{
  METARES <- NULL
  if (is.null(phenolistS))
  {
    for (k in 1:nrow(ll))
    {
      
      beta <- unlist(sapply(phenolist,function(x){x$beta_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se <- unlist(sapply(phenolist,function(x){x$se_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      N_CASES <- unlist(sapply(phenolist,function(x){x$N_CASES[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      N_CNTL <- unlist(sapply(phenolist,function(x){x$N_CNTL[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      TOTVAR_CNTL <- unlist(sapply(phenolist,function(x){x$TOTVAR_CNTL[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      TOTVAR_CASES <- unlist(sapply(phenolist,function(x){x$TOTVAR_CASES[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      beta_syno <- unlist(sapply(phenolist,function(x){x$beta_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se_syno <- unlist(sapply(phenolist,function(x){x$se_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      if (length(beta) > 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))
        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(N_CASES),sum(N_CNTL),sum(TOTVAR_CASES),sum(TOTVAR_CNTL),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,NA,NA,NA,NA,NA,NA,NA,NA)

        METARES <- rbind(METARES,METAFIN)

      }
    }
  }
  else
  {
    for (k in 1:nrow(ll))
    {

      METAFIN <- NULL

      beta <- unlist(sapply(phenolist,function(x){x$beta_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se <- unlist(sapply(phenolist,function(x){x$se_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      N_CASES <- unlist(sapply(phenolist,function(x){x$N_CASES[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      N_CNTL <- unlist(sapply(phenolist,function(x){x$N_CNTL[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      TOTVAR_CNTL <- unlist(sapply(phenolist,function(x){x$TOTVAR_CNTL[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      TOTVAR_CASES <- unlist(sapply(phenolist,function(x){x$TOTVAR_CASES[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      beta_syno <- unlist(sapply(phenolist,function(x){x$beta_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      se_syno <- unlist(sapply(phenolist,function(x){x$se_syno[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      pSKAT <- unlist(sapply(phenolistS,function(x){x$p_x[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      pSKATO <- unlist(sapply(phenolistS,function(x){x$SKATO_PVAL[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      betaBURDEN <- unlist(sapply(phenolistS,function(x){x$co_burden[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))
      seBURDEN <- unlist(sapply(phenolistS,function(x){x$se_burden[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))


      SKATN <- unlist(sapply(phenolistS,function(x){x$nmarker[x$geneset_name==ll[k,1] & x$freq_thsld==ll[k,2] & x$dfilter==ll[k,3]]}))

      pSKAT[pSKAT==0] <- 1e-16
      pSKATO[pSKATO==0] <- 1e-16
      pSKAT[pSKAT>1] <- NA
      pSKATO[pSKATO>1] <- NA

      pSKAT <- pSKAT[!is.na(pSKAT)]
      if(length(pSKATO)>1)
      {pSKATO <- pSKATO[!is.na(pSKATO)]}

      if (length(beta) > 0 & length(pSKAT) == 0 & length(pSKATO) == 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(N_CASES),sum(N_CNTL),sum(TOTVAR_CASES),sum(TOTVAR_CNTL),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,NA,NA,NA,NA,NA,NA,NA,NA)

      }
     
      if (length(beta) > 0 & length(pSKAT) > 0 & length(pSKATO) > 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))

        modburden <- metagen(betaBURDEN,seBURDEN,studlab=seq(1:length(betaBURDEN)))

        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        if (length(pSKATO) > 1)
        {modskatO <- sumlog(pSKATO)$p}
        else
        {modskatO <- pSKATO}
        

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(N_CASES),sum(N_CNTL),sum(TOTVAR_CASES),sum(TOTVAR_CNTL),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,modskat,modskatO,modburden$TE.fixed,modburden$seTE.fixed,modburden$pval.fixed,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),paste0(length(pSKATO),"/",length(phenolistS)))
      }

      if (length(beta) > 0 & length(pSKAT) > 0 & length(pSKATO) == 0)
      {
        mod <- metagen(beta,se,studlab=seq(1:length(beta)))
        modsyno <- metagen(beta_syno,se_syno,studlab=seq(1:length(beta_syno)))

        modburden <- metagen(betaBURDEN,seBURDEN,studlab=seq(1:length(betaBURDEN)))


        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),sum(N_CASES),sum(N_CNTL),sum(TOTVAR_CASES),sum(TOTVAR_CNTL),mod$TE.fixed,mod$seTE.fixed,mod$lower.fixed,mod$upper.fixed,mod$pval.fixed,modsyno$TE.fixed,modsyno$seTE.fixed,modsyno$lower.fixed,modsyno$upper.fixed,modsyno$pval.fixed,modskat,NA,modburden$TE.fixed,modburden$seTE.fixed,modburden$pval.fixed,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),NA)

      }

      if (length(beta) == 0 & length(pSKAT) > 0 & length(pSKATO) > 0)
      {

        modburden <- metagen(betaBURDEN,seBURDEN,studlab=seq(1:length(betaBURDEN)))

        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        if (length(pSKATO) > 1)
        {modskatO <- sumlog(pSKATO)$p}
        else
        {modskatO <- pSKATO}

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,modskat,modskatO,modburden$TE.fixed,modburden$seTE.fixed,modburden$pval.fixed,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),paste0(length(pSKATO),"/",length(phenolistS)))
      }

      if (length(beta) == 0 & length(pSKAT) > 0 & length(pSKATO) == 0)

      {

        if (length(pSKAT) > 1)
        {modskat <- sumlog(pSKAT)$p}
        else
        {modskat <- pSKAT}

        METAFIN <- c(as.character(ll[k,1]),as.character(ll[k,2]),as.character(ll[k,3]),NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,modskat,NA,NA,NA,NA,max(as.numeric(SKATN)),paste0(length(pSKAT),"/",length(phenolistS)),NA)

      }

        METARES <- rbind(METARES,METAFIN)
    }
  }
colnames(METARES) <- c("geneset_name","freq_thsld","dfilter","N_CASES","N_CNTL","TOTVAR_CASES","TOTVAR_CNTL","beta","se","lower_CI","upper_CI","pval","beta_syno","se_syno","lower_CI_syno","upper_CI_syno","pval_syno","SKAT_P","SKATO_P","BURDEN_BETA","BURDEN_SE","BURDEN_PVAL","SKAT_NVAR","N_studies_SKAT","N_studies_SKATO")
return(METARES)
}



### T2D ###
print("T2D")

phenolist <- list(diabD_afrB[diabD_afrB$pheno=="T2D",],diabD_amrB[diabD_amrB$pheno=="T2D",],diabD_easB[diabD_easB$pheno=="T2D",],diabD_nfeB[diabD_nfeB$pheno=="T2D",],diabD_finB[diabD_finB$pheno=="T2D",],diabD_sasB[diabD_sasB$pheno=="T2D",],migenD_afrB[migenD_afrB$pheno=="T2D_BASELINE",],migenD_nfeB[migenD_nfeB$pheno=="T2D_BASELINE",],migenD_sasB[migenD_sasB$pheno=="T2D_BASELINE",])
phenolistS <- list(SdiabD_afrB[SdiabD_afrB$pheno=="T2D",],SdiabD_amrB[SdiabD_amrB$pheno=="T2D",],SdiabD_easB[SdiabD_easB$pheno=="T2D",],SdiabD_nfeB[SdiabD_nfeB$pheno=="T2D",],SdiabD_finB[SdiabD_finB$pheno=="T2D",],SdiabD_sasB[SdiabD_sasB$pheno=="T2D",],SmigenD_afrB[SmigenD_afrB$pheno=="T2D_BASELINE",],SmigenD_nfeB[SmigenD_nfeB$pheno=="T2D_BASELINE",],SmigenD_sasB[SmigenD_sasB$pheno=="T2D_BASELINE",])

METAT2D <- METAD(phenolist,phenolistS,ll)


### IBD ###
print("IBD")

phenolist <- list(ibdD_nfeB[ibdD_nfeB$pheno=="IBD",],ibdD_finB[ibdD_finB$pheno=="IBD",],ibdD_askB[ibdD_askB$pheno=="IBD",])
phenolistS <- list(SibdD_nfeB[SibdD_nfeB$pheno=="IBD",],SibdD_finB[SibdD_finB$pheno=="IBD",],SibdD_askB[SibdD_askB$pheno=="IBD",])

METAIBD <- METAD(phenolist,phenolistS,ll)


### UC ###
print("UC")

phenolist <- list(ibdD_nfeB[ibdD_nfeB$pheno=="UC",],ibdD_finB[ibdD_finB$pheno=="UC",],ibdD_askB[ibdD_askB$pheno=="UC",])
phenolistS <- list(SibdD_nfeB[SibdD_nfeB$pheno=="UC",],SibdD_finB[SibdD_finB$pheno=="UC",],SibdD_askB[SibdD_askB$pheno=="UC",])

METAUC <- METAD(phenolist,phenolistS,ll)



### CD ###
print("CD")

phenolist <- list(ibdD_finB[ibdD_finB$pheno=="CD",],ibdD_nfeB[ibdD_nfeB$pheno=="CD",],ibdD_askB[ibdD_askB$pheno=="CD",])
phenolistS <- list(SibdD_finB[SibdD_finB$pheno=="CD",],SibdD_nfeB[SibdD_nfeB$pheno=="CD",],SibdD_askB[SibdD_askB$pheno=="CD",])

METACD <- METAD(phenolist,phenolistS,ll)



### Early ONSET MI ##
print("EOMI")

phenolist <- list(migenD2_afrB[migenD2_afrB$pheno=="EOMI",],migenD2_nfeB[migenD2_nfeB$pheno=="EOMI",],migenD2_sasB[migenD2_sasB$pheno=="EOMI",])
phenolistS <- list(SmigenD2_afrB[SmigenD2_afrB$pheno=="EOMI",],SmigenD2_nfeB[SmigenD2_nfeB$pheno=="EOMI",],SmigenD2_sasB[SmigenD2_sasB$pheno=="EOMI",])

METAEOMI <- METAD(phenolist,phenolistS,ll)


### CHD ###
print("CHD")

phenolist <- list(migenD2_afrB[migenD2_afrB$pheno=="CHD",],migenD2_nfeB[migenD2_nfeB$pheno=="CHD",],migenD2_sasB[migenD2_sasB$pheno=="CHD",])
phenolistS <- list(SmigenD2_afrB[SmigenD2_afrB$pheno=="CHD",],SmigenD2_nfeB[SmigenD2_nfeB$pheno=="CHD",],SmigenD2_sasB[SmigenD2_sasB$pheno=="CHD",])

METACHD <- METAD(phenolist,phenolistS,ll)


### ID ###
print("ID")

phenolist <- list(nfidD_finB[nfidD_finB$pheno=="intd",])
phenolistS <- list(SnfidD_finB[SnfidD_finB$pheno=="intd",])

METAID <- METAD(phenolist,phenolistS,ll)


### SCZ ###
print("SCZ")

phenolist <- list(sweD2_nfeB[sweD2_nfeB$pheno=="scz",],dbsD_nfeB[dbsD_nfeB$pheno=="SCZ",])
phenolistS <- list(SsweD2_nfeB[SsweD2_nfeB$pheno=="scz",],SdbsD_nfeB[SdbsD_nfeB$pheno=="SCZ",])


METASCZ <- METAD(phenolist,phenolistS,ll)

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="SCZ",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="SCZ",])


METASCZDBS <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="SCZ_PURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="SCZ_PURE",])


METASCZP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="SCZ_UNPURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="SCZ_UNPURE",])


METASCZNP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="SCZ_UNPUREID",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="SCZ_UNPUREID",])


METASCZNPID <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD3_nfeB[dbsD3_nfeB$pheno=="SCZ",])
phenolistS <- list(SdbsD3_nfeB[SdbsD3_nfeB$pheno=="SCZ",])


METASCZFEMALE <- METAD(phenolist,NULL,ll)

phenolist <- list(dbsD4_nfeB[dbsD4_nfeB$pheno=="SCZ",])
phenolistS <- list(SdbsD4_nfeB[SdbsD4_nfeB$pheno=="SCZ",])


METASCZMALE <- METAD(phenolist,NULL,ll)


### AUTISM ###
print("AUTISM")

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="AUTISM",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="AUTISM",])


METAAUTISM <- METAD(phenolist,phenolistS,ll)

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="AUTISM_PURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="AUTISM_PURE",])


METAAUTISMP <- METAD(phenolist,NULL,ll)

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="AUTISM_UNPURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="AUTISM_UNPURE",])


METAAUTISMNP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="AUTISM_UNPUREID",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="AUTISM_UNPUREID",])


METAAUTISMNPID <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD3_nfeB[dbsD3_nfeB$pheno=="AUTISM",])
phenolistS <- list(SdbsD3_nfeB[SdbsD3_nfeB$pheno=="AUTISM",])


METAAUTISMFEMALE <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD4_nfeB[dbsD4_nfeB$pheno=="AUTISM",])
phenolistS <- list(SdbsD4_nfeB[SdbsD4_nfeB$pheno=="AUTISM",])


METAAUTISMMALE <- METAD(phenolist,NULL,ll)




### ADHD ###
print("ADHD")

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="ADHD",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="ADHD",])


METAADHD <- METAD(phenolist,phenolistS,ll)

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="ADHD_PURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="ADHD_PURE",])

METAADHDP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="ADHD_UNPURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="ADHD_UNPURE",])

METAADHDNP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="ADHD_UNPUREID",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="ADHD_UNPUREID",])

METAADHDNPID <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD3_nfeB[dbsD3_nfeB$pheno=="ADHD",])
phenolistS <- list(SdbsD3_nfeB[SdbsD3_nfeB$pheno=="ADHD",])

METAADHDFEMALE <- METAD(phenolist,NULL,ll)

phenolist <- list(dbsD4_nfeB[dbsD4_nfeB$pheno=="ADHD",])
phenolistS <- list(SdbsD4_nfeB[SdbsD4_nfeB$pheno=="ADHD",])


METAADHDMALE <- METAD(phenolist,NULL,ll)



### BP ####
print("BP")

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="BP",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="BP",])


METABP <- METAD(phenolist,phenolistS,ll)

phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="BP_PURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="BP_PURE",])

METABPP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="BP_UNPURE",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="BP_UNPURE",])

METABPNP <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD_nfeB[dbsD_nfeB$pheno=="BP_UNPUREID",])
phenolistS <- list(SdbsD_nfeB[SdbsD_nfeB$pheno=="BP_UNPUREID",])

METABPNPID <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD3_nfeB[dbsD3_nfeB$pheno=="BP",])
phenolistS <- list(SdbsD3_nfeB[SdbsD3_nfeB$pheno=="BP",])

METABPFEMALE <- METAD(phenolist,NULL,ll)


phenolist <- list(dbsD4_nfeB[dbsD4_nfeB$pheno=="BP",])
phenolistS <- list(SdbsD4_nfeB[SdbsD4_nfeB$pheno=="BP",])

METABPMALE <- METAD(phenolist,NULL,ll)


save(list = ls(all.names = TRUE)[grep("^META",ls(all.names = TRUE))], file = paste0(path,"all_results_association.Rdata"))